In [1]:
from __future__ import print_function    # (at top of module)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pprint
import time # time.sleep(5)
import csv

In [2]:
import pandas as pd
import numpy as np
import scipy
from skimage import io
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import decomposition
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal as mvn

In [3]:
def delete_available_markets(spotipy_obj):
    ''' Quick function to remove available_markets.
    '''
    for _d in spotipy_obj:
        _d.pop('available_markets') # who cares?  Adds clutter so let's remove the key.
    return spotipy_obj

In [4]:
def write_list_of_lists(file_path_name_ext, list_of_lists, column_headers):
    with open(file_path_name_ext,'w',newline="", encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(column_headers)
        csv_writer.writerows(list_of_lists)
    return True
        

In [5]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [29]:
riaa = pd.read_csv('data/RIAA_Top_Artists_Cleaned.csv')
column_name = [row.lower().replace(' ','_') for row in riaa.columns.values]
riaa.columns = column_name

In [ ]:
# artist_obj[0].keys() 
# dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])
artist_obj = sp.search(riaa.artist[0],type='artist')['artists']['items']

In [298]:
with open('data/spotify_artists.csv','r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    _artists = [row for row in csv_reader]

artist_uris = [row[3] for row in _artists]

artist_skip_list = [row[0].upper() for row in _artists]

new_artist_list = []
for artist in riaa.artist.values:
    if artist not in artist_skip_list:
        new_artist_list.append(artist)

In [299]:
with open('data/spotify_artist_top_tracks.csv','r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    _tracks = [row for row in csv_reader]

skip_artist_uris_for_tracks = [row[0] for row in _tracks]

new_artist_list_for_tracks = []

for artist in artist_uris:
    if artist not in skip_artist_uris_for_tracks:
        new_artist_list_for_tracks.append(artist)

In [313]:
with open('data/spotify_artist_related_artists.csv','r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    _related = [row for row in csv_reader]

skip_artist_uris_for_related = [row[0] for row in _related]

new_artist_list_for_related = []

for artist in artist_uris:
    if artist not in skip_artist_uris_for_related:
        new_artist_list_for_related.append(artist)

In [ ]:
spotify_artist_related_artists

In [173]:
for artist in new_artist_list:
    artist_obj = sp.search(artist,type='artist')['artists']['items']
    try:
        followers = artist_obj[0]['followers']['total']
    except IndexError:
        followers = ''
    try:
        genres = '; '.join(artist_obj[0]['genres'])
    except IndexError:
        genres=''
    try:
        name = artist_obj[0]['name']
    except IndexError:
        name=''
    try:
        popularity = artist_obj[0]['popularity']
    except IndexError:
        popularity=''
    try:
        _artist_uri = artist_obj[0]['uri']
    except IndexError:
        _artist_uri = ''
    _artists.append([name, popularity, followers,genres, _artist_uri  ])
    time.sleep(0.25) # Trying not to make spotify mad.

In [174]:
column_headers = ['name', 'popularity', 'followers','genres', '_artist_uri'  ]
write_list_of_lists('data/spotify_artists.csv',_artists)

True

In [82]:
# name artist_obj[0]['name']
# Followers artist_obj[0]['followers']['total']
# popularity artist_obj[0]['popularity']
artist_uri = artist_obj[0]['uri']

In [181]:
len(artist_uris)

2143

In [185]:
top_tracks = sp.artist_top_tracks('spotify:artist:3WrFJ7ztbogyGnTHbHJFl2')

In [193]:
top_tracks['tracks'][0].keys() # dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [203]:
albumn_name = top_tracks['tracks'][0]['album']['name']
albumn_release_date = top_tracks['tracks'][0]['album']['release_date']
albumn_total_tracks = top_tracks['tracks'][0]['album']['total_tracks']
top_tracks['tracks'][0]['duration_ms']
top_tracks['tracks'][0]['explicit']
track_name = top_tracks['tracks'][0]['name']
track_popularity = top_tracks['tracks'][0]['popularity']
track_uri = top_tracks['tracks'][0]['uri']

In [213]:
_tracks =[]

In [301]:

for artist_uri in new_artist_list_for_tracks:
    # print(artist_uri)
    for track in sp.artist_top_tracks(artist_uri)['tracks']:
        
        albumn_name = track['album']['name']
        albumn_release_date = track['album']['release_date']
        albumn_total_tracks = track['album']['total_tracks']
        duration_ms = track['duration_ms']
        explicit = track['explicit']
        track_name = track['name']
        track_popularity = track['popularity']

        track_uri = track['uri']

        for feature in sp.audio_features(track_uri):
            try:
                danceability = feature['danceability']
            except TypeError:
                danceability =''
            try:
                energy = feature['energy']
            except TypeError:
                energy =''
            key = feature['key']
            loudness = feature['loudness']
            mode = feature['mode']
            speechiness = feature['speechiness']
            acousticness = feature['acousticness']
            instrumentalness = feature['instrumentalness']
            liveness = feature['liveness']
            valence = feature['valence']
            tempo = feature['tempo']
            time_signature = feature['time_signature']
            
            _tracks.append([artist_uri,albumn_name,albumn_release_date,albumn_total_tracks,duration_ms,explicit, track_name,track_popularity,track_uri, danceability, energy,key, loudness,mode,speechiness, acousticness,instrumentalness,liveness,valence, tempo,time_signature])
    
    time.sleep(0.25) # Trying not to make spotify mad.

In [302]:
column_headers = ['artist_uri','albumn_name','albumn_release_date','albumn_total_tracks','duration_ms','explicit', 'track_name','track_popularity','track_uri', 'danceability', 'energy','key', 
                  'loudness','mode','speechiness', 'acousticness','instrumentalness','liveness','valence', 'tempo','time_signature']
write_list_of_lists('data/spotify_artist_top_tracks.csv',_tracks,column_headers)

True

In [314]:

for node_artist in new_artist_list_for_related:
    artist_related_artist_obj = sp.artist_related_artists(node_artist)['artists']
    for artist in artist_related_artist_obj:
        edge_artist = artist['name']
        followers = artist['followers']['total']
        genres = '; '.join(artist['genres'])
        popularity = artist['popularity']
        _type = artist['type']

        _related.append([node_artist,edge_artist,followers,genres,popularity,_type ])
    time.sleep(0.25) # Trying not to make spotify mad.

In [315]:
column_headers = ['node_artist','edge_artist','followers','genres','popularity','_type']

write_list_of_lists('data/spotify_artist_related_artists.csv',_related,column_headers)

True

In [ ]:
_related

In [83]:
albums_obj = sp.artist_albums(artist_uri)['items']


In [89]:
# albums_obj[0]['release_date']
# albums_obj[0]['total_tracks']
# albums_obj[0]['type']
# albums_obj[0]['album_group']
# albums_obj[0]['album_type']
# albums_obj[0]['artists'][0]['name'] # albumn atrist name (Collab??)
artist_albumn_uri = albums_obj[0]['uri']

In [94]:
# albumn_tracks_obj[0].keys() dict_keys(['artists', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])
albumn_tracks_obj = delete_available_markets(sp.album_tracks(artist_albumn_uri)['items'])

In [97]:
albumn_tracks_obj[0].keys() # dict_keys(['artists', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

dict_keys(['artists', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

In [100]:
# albumn_tracks_obj[0]['disc_number']
# albumn_tracks_obj[0]['duration_ms']
# albumn_tracks_obj[0]['explicit']
# albumn_tracks_obj[0]['id']
# albumn_tracks_obj[0]['name']
# albumn_tracks_obj[0]['track_number']
# albumn_tracks_obj[0]['type']
track_uri = albumn_tracks_obj[0]['uri']

In [103]:
track_features = sp.audio_features(track_uri)

In [112]:
# dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
# track_features[0]['danceability']
# track_features[0]['energy']
# track_features[0]['key']
# track_features[0]['loudness']
# track_features[0]['mode']
# track_features[0]['speechiness']
# track_features[0]['acousticness']

# track_features[0]['instrumentalness']
# track_features[0]['liveness']
# track_features[0]['valence']
# track_features[0]['tempo']
# track_features[0]['time_signature']
track_analysis_url = track_features[0]['id']

In [115]:
track_features[0]

{'danceability': 0.628,
 'energy': 0.524,
 'key': 2,
 'loudness': -11.84,
 'mode': 1,
 'speechiness': 0.0773,
 'acousticness': 0.325,
 'instrumentalness': 0.0995,
 'liveness': 0.195,
 'valence': 0.272,
 'tempo': 124.176,
 'type': 'audio_features',
 'id': '4bVT9bpThKnkFtl1xeB2bh',
 'uri': 'spotify:track:4bVT9bpThKnkFtl1xeB2bh',
 'track_href': 'https://api.spotify.com/v1/tracks/4bVT9bpThKnkFtl1xeB2bh',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4bVT9bpThKnkFtl1xeB2bh',
 'duration_ms': 283373,
 'time_signature': 4}

In [116]:
# track_an = sp.audio_analysis(track_analysis_url)
# track_an

In [101]:

#  # 'album',
#  # 'album_tracks',
#  # 'albums',
#  # 'artist',
#  # 'artist_albums',
#  # 'artist_related_artists',
#  # 'artist_top_tracks',
#  # 'artists',
#  # 'audio_analysis',
#  # 'audio_features',


# dir(sp)

In [16]:
artist_obj = sp.search(_need_art[0],type='artist')['artists']['items']

In [33]:
_list =[]
for r_art in _need_art:
    artist_obj = sp.search(r_art,type='artist')['artists']['items']
    cnt =0 
    for artist in artist_obj:
        _list.append([r_art, artist['name'],
        artist['followers']['total'],
        artist['popularity'],
        artist['uri']
        ]
        )
        cnt+=1
        if cnt >=3:
            break
    time.sleep(0.25) # Trying not to make spotify mad.

In [29]:
column_headers = ['source','spotify_artist','followers','popularity','uri']
write_list_of_lists('data/remaining_artists.csv',_list,column_headers)

True

In [36]:
_need_top_tracks = [
'spotify:artist:4eQeZFlDU3froIAkzS6vnC',
'spotify:artist:0SzEp7uVyzBozMmx2zx6TB',
'spotify:artist:0wFCfi4dqZB5DO7sAQpQUZ',
'spotify:artist:0Xf8oDAJYd2D0k3NLI19OV',
'spotify:artist:0zyyO1rbhrlJmPVXbOJGqZ',
'spotify:artist:2aS6jYh7ysTL1ZUsHneNgM',
'spotify:artist:2HMuzFmTtwL4ur1zJNCU7f',
'spotify:artist:2S8UlyXW4JjjHcbeg8ddIo',
'spotify:artist:3DmG65yHQsMms7WAvrZOdt',
'spotify:artist:3ICyfoySNDZqtBVmaBT84I',
'spotify:artist:3R0dLbQvI6x16t2bLaE9wE',
'spotify:artist:3vbKDsSS70ZX9D2OcvbZmS',
'spotify:artist:3zTOe1BtyTkwNvYZOxXktX',
'spotify:artist:4jtcZjpuoSYBWP8xoXGi2P',
'spotify:artist:4LjrTfWcvqeHDcH6kr2Msd',
'spotify:artist:4UUA0PUMRTJsfa3eqQGY0T',
'spotify:artist:51FyBBOW464toW099BsW8z',
'spotify:artist:5g2n2H9ndqvejERtwZ1rHh',
'spotify:artist:6819eS3ZHurH0zed5dtPxj',
'spotify:artist:6IE6z7DcZIT4Ml3Fh5Ivch',
'spotify:artist:6lDJOI3SbOaTNvQsywhoz9',
'spotify:artist:6VOkIa4pRNEdugeOdePtRp',
'spotify:artist:77yIRk1rQ3x2bFWzvL6ri7',
'spotify:artist:7dXeJ8kAqTqtvNWQNV3sdU',
]

In [39]:
_tracks = []
for artist_uri in _need_top_tracks:
    # print(artist_uri)
    for track in sp.artist_top_tracks(artist_uri)['tracks']:
        
        albumn_name = track['album']['name']
        albumn_release_date = track['album']['release_date']
        albumn_total_tracks = track['album']['total_tracks']
        duration_ms = track['duration_ms']
        explicit = track['explicit']
        track_name = track['name']
        track_popularity = track['popularity']

        track_uri = track['uri']

        for feature in sp.audio_features(track_uri):
            try:
                danceability = feature['danceability']
            except TypeError:
                danceability =''
            try:
                energy = feature['energy']
            except TypeError:
                energy =''
            key = feature['key']
            loudness = feature['loudness']
            mode = feature['mode']
            speechiness = feature['speechiness']
            acousticness = feature['acousticness']
            instrumentalness = feature['instrumentalness']
            liveness = feature['liveness']
            valence = feature['valence']
            tempo = feature['tempo']
            time_signature = feature['time_signature']
            
            _tracks.append([artist_uri,albumn_name,albumn_release_date,albumn_total_tracks,duration_ms,explicit, track_name,track_popularity,track_uri, danceability, energy,key, loudness,mode,speechiness, acousticness,instrumentalness,liveness,valence, tempo,time_signature])
    
    time.sleep(0.25) # Trying not to make spotify mad.

In [40]:
column_headers = ['artist_uri','albumn_name','albumn_release_date','albumn_total_tracks','duration_ms','explicit', 'track_name','track_popularity','track_uri', 'danceability', 'energy','key', 
                  'loudness','mode','speechiness', 'acousticness','instrumentalness','liveness','valence', 'tempo','time_signature']
write_list_of_lists('data/spotify_artist_top_tracks2.csv',_tracks,column_headers)

True

In [6]:
_related_art = [
'spotify:artist:7urq0VfqxEYEEiZUkebXT4',
'spotify:artist:3DmG65yHQsMms7WAvrZOdt',
'spotify:artist:3zTOe1BtyTkwNvYZOxXktX',
'spotify:artist:2HMuzFmTtwL4ur1zJNCU7f',
'spotify:artist:41Q0HrwWBtuUkJc7C1Rp6K',
'spotify:artist:51FyBBOW464toW099BsW8z',
'spotify:artist:2aS6jYh7ysTL1ZUsHneNgM',
'spotify:artist:2S8UlyXW4JjjHcbeg8ddIo',
'spotify:artist:6819eS3ZHurH0zed5dtPxj',
'spotify:artist:3ICyfoySNDZqtBVmaBT84I',
'spotify:artist:6IE6z7DcZIT4Ml3Fh5Ivch',
'spotify:artist:6VOkIa4pRNEdugeOdePtRp',
'spotify:artist:7dXeJ8kAqTqtvNWQNV3sdU',
'spotify:artist:3vbKDsSS70ZX9D2OcvbZmS',
'spotify:artist:5g2n2H9ndqvejERtwZ1rHh',
'spotify:artist:4LjrTfWcvqeHDcH6kr2Msd',
'spotify:artist:0Xf8oDAJYd2D0k3NLI19OV',
'spotify:artist:4ghjRm4M2vChDfTUycx0Ce',
'spotify:artist:5rSXSAkZ67PYJSvpUpkOr7',
'spotify:artist:0xOeVMOz2fVg5BJY3N6akT',
'spotify:artist:5S6hjAxgxjsLylsTtMIimO',
'spotify:artist:56dO9zeHKuU5Gvfc2kxHNw',
'spotify:artist:7jefIIksOi1EazgRTfW2Pk',
'spotify:artist:4IR8ZkpbyyKrqsIzchF3NB',
'spotify:artist:4nCzT3o7lMgCWBcmdwCMsT',
'spotify:artist:2MqhkhX4npxDZ62ObR5ELO',
'spotify:artist:0niJkG4tKkne3zwr7I8n9n',
'spotify:artist:1UdQqCUR7RwB9YYJONwbdM',
'spotify:artist:6Uhp7WA6sjm5ZL6Xz561de',
'spotify:artist:1nSRa2YjjFWJLHGKM07oGQ',
'spotify:artist:6LqNN22kT3074XbTVUrhzX',
'spotify:artist:2MyNAoQL07EABerr6yhoT4',
'spotify:artist:7iUhmKPNkkPPS6FCQxqtNq',
'spotify:artist:2yEwvVSSSUkcLeSTNyHKh8',
'spotify:artist:2WKdxPFRD7IqZvlIAvhMgY',
'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG',
'spotify:artist:50co4Is1HCEo8bhOyUWKpn',
'spotify:artist:5dHt1vcEm9qb8fCyLcB3HL',
'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY',
'spotify:artist:6BMQo3hxi4qwLiZpHKK5iL',
'spotify:artist:2Hjj68yyUPiC0HKEOigcEp',
'spotify:artist:21gGD9dUby5ls0belA1wqZ',
'spotify:artist:1fvz0vd4P0LNMkAysF1ivk',
'spotify:artist:7oHzn7edwmrYClrPRINkbn',
'spotify:artist:7BLOa6FxCbMnowidkQfLIA',
'spotify:artist:6MLDcHrNh4OqxDZAjMt5pt',
'spotify:artist:7vWlb4pM85jCHvV771qZZW',
'spotify:artist:7uWXomGkKtIq9B4XgA3gch',
'spotify:artist:5aBxFPaaGk9204ssHUvXWN',
'spotify:artist:5IcR3N7QB1j6KBL8eImZ8m',
'spotify:artist:0MJ8sSwKrcYenpNAUeImuU',
'spotify:artist:2cnMpRsOVqtPMfq7YiFE6K',
'spotify:artist:5DWj9QLyCN9dyQydd1P9QG',
'spotify:artist:6kBjAFKyd0he7LiA5GQ3Gz',
'spotify:artist:6KO6G41BBLTDNYOLefWTMU',
'spotify:artist:540vIaP2JwjQb9dm3aArA4',
'spotify:artist:1vCWHaC5f2uS3yhpwWbIA6',
'spotify:artist:711MCceyCBcFnzjGY4Q7Un',
'spotify:artist:5CiGnKThu5ctn9pBxv7DGa',
'spotify:artist:5v8jlSmAQfrkTjAlpUfWtu',
'spotify:artist:4sj6D0zlMOl25nprDJBiU9',
'spotify:artist:7tjbDPvrdvDshcpEMXKRVb',
'spotify:artist:2m62cc253Xvd9qYQ8d2X3d',
'spotify:artist:7t0rwkOPGlDPEhaOcVtOt9',
'spotify:artist:0epOFNiUfyON9EYx7Tpr6V',
'spotify:artist:2nszmSgqreHSdJA3zWPyrW',
'spotify:artist:2wY79sveU1sp5g7SokKOiI',
'spotify:artist:2qxJFvFYMEDqd7ui6kSAcq',
'spotify:artist:07XSN3sPlIlB2L2XNcTwJw',
'spotify:artist:5MspMQqdVbdwP6ax3GXqum',
'spotify:artist:37394IP6uhnjIpsawpMu4l',
'spotify:artist:3LkSiHbjqOHCKCqBfEZOTv',
'spotify:artist:695W5F2Ih8dYahLdjVOIoH',
'spotify:artist:46njgd2Rq9tZc4ZjeQMgbh',
'spotify:artist:3iDD7bnsjL9J4fO298r0L0',
'spotify:artist:53XhwfbYqKCa1cC15pYq2q',
'spotify:artist:3NPpFNZtSTHheNBaWC82rB',
'spotify:artist:6lHL3ubAMgSasKjNqKb8HF',
'spotify:artist:7J2lZBANizgPNfUzux31PV',
'spotify:artist:5nGIFgo0shDenQYSE0Sn7c',
'spotify:artist:0mkcYaXUbJUs5mJointuzB',
'spotify:artist:1LZEQNv7sE11VDY3SdxQeN',
'spotify:artist:17XXKfRBMCWvLrqGoNkJXm',
'spotify:artist:24Oiw7BlvO1BETecDLJt6m',
'spotify:artist:6FBDaR13swtiWwGhX1WQsP',
'spotify:artist:1EemADz12kjlGwkpY4EINv',
'spotify:artist:5ILrArfIV0tMURcHJN8Q07',
'spotify:artist:5zzrJD2jXrE9dZ1AklRFcL',
'spotify:artist:1zuJe6b1roixEKMOtyrEak',
'spotify:artist:3UVRliakQfa1pMWIsNuiZ8',
'spotify:artist:0ZrpamOxcZybMHGg1AYtHP',
'spotify:artist:2AOt5htsbtyaHd5Eq3kl3j',
'spotify:artist:6Wr3hh341P84m3EI8qdn9O',
'spotify:artist:5DgjOwTN6o76J5Gf8MzEoL',
'spotify:artist:4FtSnMlCVxCswABUmdhwpm',
'spotify:artist:2hl0xAkS2AIRAu23TVMBG1',
'spotify:artist:3rIZMv9rysU7JkLzEaC5Jp',
'spotify:artist:4gwpcMTbLWtBUlOijbVpuu',
'spotify:artist:6DIS6PRrLS3wbnZsf7vYic',
'spotify:artist:1zK4ACgLi1lVPpfmmcwOTh',
'spotify:artist:2d0hyoQ5ynDBnkvAbJKORj',
'spotify:artist:3YQKmKGau1PzlVlkL1iodx',
'spotify:artist:5OSQDUNmll299ldFIF1Dau',
'spotify:artist:20qISvAhX20dpIbOOzGK3q',
'spotify:artist:5yG7ZAZafVaAlMTeBybKAL',
'spotify:artist:0ecnb1WPePfrHg4yfQfmqN',
'spotify:artist:5jWuVnPLmjABrgCGianzs9',
'spotify:artist:3aQeKQSyrW4qWr35idm0cy',
'spotify:artist:0z4gvV4rjIZ9wHck67ucSV',
'spotify:artist:6x33CmZWo2Ve4hxYl2Craq',
'spotify:artist:7okwEbXzyT2VffBmyQBWLz',
'spotify:artist:6Ff53KvcvAj5U7Z1vojB5o',
'spotify:artist:60CFNa0ggWjV8OSDey8aGT',
'spotify:artist:26bcq2nyj5GB7uRr558iQg',
'spotify:artist:4LG4Bs1Gadht7TCrMytQUO',
'spotify:artist:3YbOSxo85kla7RID8ugnW3',
'spotify:artist:3hYtANQYrE6pd2PbtEyTIy',
'spotify:artist:6OheJTrDFGiyZ67F1BBLhc',
'spotify:artist:3hv9jJF3adDNsBSIQDqcjp',
'spotify:artist:6LuN9FCkKOj5PcnpouEgny',
'spotify:artist:4zKhX6hYCQPNs406umP9Fx',
'spotify:artist:4jWzNq3sdzSpqn7BnzvIKn',
'spotify:artist:5pnbUBPifNnlusY8kTBivi',
'spotify:artist:64ovxBmjJ9ZDqkzEXVWpVc',
'spotify:artist:0a0l3QVhfMwQNAO4wPAmP9',
'spotify:artist:6IYnSXO40Bh7Zdqhf6rQoj',
'spotify:artist:2SHhfs4BiDxGQ3oxqf0UHY',
'spotify:artist:2BTZIqw0ntH9MvilQ3ewNY',
'spotify:artist:62sPt3fswraiEPnKQpAbdE',
'spotify:artist:45lorWzrKLxfKlWpV7r9CN',
'spotify:artist:7jmTilWYlKOuavFfmQAcu6',
'spotify:artist:4e5V1Q2dKCzbLVMQ8qbTn6',
'spotify:artist:6qXwLwTLdA44HYsA26vaNU',
'spotify:artist:6kXp61QMZFPcKMcRPqoiVj',
'spotify:artist:27XI002zivG9xuzrZ9dIb1',
'spotify:artist:0dhxm9QdU7nzflPS6xGQ8P',
'spotify:artist:0hCNtLu0JehylgoiP8L4Gh',
'spotify:artist:4kYSro6naA4h99UJvo89HB',
'spotify:artist:0LIll5i3kwo5A3IDpipgkS',
'spotify:artist:7HQH1SJokcVMdstilKJ2S8',
'spotify:artist:1stGNsBocVFM9MfQtySETw',
'spotify:artist:4NgfOZCL9Ml67xzM0xzIvC',
'spotify:artist:4Otx4bRLSfpah5kX8hdgDC',
'spotify:artist:4toEjJSZu1rbfX2hfVdZFA',
'spotify:artist:6jHG1YQkqgojdEzerwvrVv',
'spotify:artist:1P8IfcNKwrkQP5xJWuhaOC',
'spotify:artist:3zpKjsMg2gw1St5WcWoUJN',
'spotify:artist:19KwjzvIL92r29IINtlPNP',
'spotify:artist:5me0Irg2ANcsgc93uaYrpb',
'spotify:artist:6O2zJ0tId7g07yzHtX0yap',
'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5',
'spotify:artist:3TVXtAsR1Inumwj472S9r4',
'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x',
'spotify:artist:0rG0AZBscc8S8q1ahIsasI',
'spotify:artist:12FHARd9fY0Tu0ila4Ua25',
'spotify:artist:5F1aAS1duwlzExnPs3l2Xe',
'spotify:artist:7xsMZdxw6eEJXGTLZulONO',
'spotify:artist:7zOcRYg1BYXpwr71XTWNr7',
'spotify:artist:7vCtweS8UVAXTyau2j0rDT',
'spotify:artist:0l5OmkXhNcXtfa1W9dTon6',
'spotify:artist:4epBFW4UHEmgjIK5xOrBhk',
'spotify:artist:0vYQRW5LIDeYQOccTviQNX',
'spotify:artist:3PhoLpVuITZKcymswpck5b',
'spotify:artist:1zJBFCev9UwOMcrZsLi2od',
'spotify:artist:2CLVPk9FcywjClBcTvWPkT',
'spotify:artist:5Qi4Bb7a8C0a00NZcA77L0',
'spotify:artist:53B8dEQzmtefvkdCAkO0YR',
'spotify:artist:14pVkFUHDL207LzLHtSA18',
'spotify:artist:0Mz5XE0kb1GBnbLQm2VbcO',
'spotify:artist:0yN7xI1blow9nYIK0R8nM7',
'spotify:artist:4ZqBwBVB3dsYWL3d5Spi10',
'spotify:artist:07xwCifj3iE7F6fYHin4TU',
'spotify:artist:0VdoLGGpWcDZnzep6SSTdP',
'spotify:artist:2lFHVcUeJ9Gq6AZiU3ZAOa',
'spotify:artist:0jJNGWrpjGIHUdTTJiIYeB',
'spotify:artist:0Grjlu7ncIuCaSYvCs9fcd',
'spotify:artist:4NiJW4q9ichVqL1aUsgGAN',
'spotify:artist:45TgXXqMDdF8BkjA83OM7z',
'spotify:artist:2rc78XDH9zuJP6bm78lU8Z',
'spotify:artist:1CD77o9fbdyQFrHnUPUEsF',
'spotify:artist:0qT79UgT5tY4yudH9VfsdT',
'spotify:artist:0ErzCpIMyLcjPiwT4elrtZ',
'spotify:artist:4r63FhuTkUYltbVAg5TQnk',
'spotify:artist:1cNDP5yjU5vjeR8qMf4grg',
'spotify:artist:3AA28KZvwAUcZuOKwyblJQ',
'spotify:artist:3E3tW69eMfAffLGgsmP3GG',
'spotify:artist:3Ubvmshojm0HJHWWr4C6GF',
'spotify:artist:3z6Uf5IZkN4TogRpRpgD84',
'spotify:artist:3Y5abCLsMxsOLaZ8rYusSR',
'spotify:artist:49e4v89VmlDcFCMyDv9wQ9',
'spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0',
'spotify:artist:5N6GwJzOcOY5kv8p0NjhYL',
'spotify:artist:0LcJLqbBmaGUft1e9Mm8HV',
'spotify:artist:6futYSDVulYR2PktBjTB5W',
'spotify:artist:1gPhS1zisyXr5dHTYZyiMe',
'spotify:artist:4salDzkGmfycRqNUbyBphh',
'spotify:artist:6kz53iCdBSqhQCZ21CoLcc',
'spotify:artist:1g1yxsNVPhMUl9GrMjEb2o',
'spotify:artist:3NRFinRTEqUCfaTTZmk8ek',
'spotify:artist:2bA6fzP0lMAQ4kz6CF61w8',
'spotify:artist:24XtlMhEMNdi822vi0MhY1',
'spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ',
'spotify:artist:0TImkz4nPqjegtVSMZnMRq',
'spotify:artist:0urTpYCsixqZwgNTkPJOJ4',
'spotify:artist:6nzxy2wXs6tLgzEtqOkEi2',
'spotify:artist:29ywwKkxfoH7iWwNY1UezA',
'spotify:artist:3fhOTtm0LBJ3Ojn4hIljLo',
'spotify:artist:3KV3p5EY4AvKxOlhGHORLg',
'spotify:artist:3tCxM1rTj2qUXsiQZxvt2I',
'spotify:artist:7dOBabd5O4CvKrg4iriHTM',
'spotify:artist:3xzv5kMXulv03oeXs6H8eL',
'spotify:artist:1PCZpxHJz7WAMF8EEq8bfc',
'spotify:artist:6w7fc6IZlo5zwBaKT5jU1X',
'spotify:artist:0nmQIMXWTXfhgOBdNzhGOs',
'spotify:artist:3jdirYL01245TvAH39GfM6',
'spotify:artist:0oea1hwGMfUxZbLxJc1XUN',
'spotify:artist:6ZjFtWeHP9XN7FeKSUe80S',
'spotify:artist:3YcBF2ttyueytpXtEzn1Za',
'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD',
'spotify:artist:1u7kkVrr14iBvrpYnZILJR',
'spotify:artist:1GRl6sRyLg9ToOohIE2wW5',
'spotify:artist:2Cd98zHVdZeOCisc6Gi2sB',
'spotify:artist:7MpUvihmfilIxyN20kXwQj',
'spotify:artist:7LCp4MN0SOIVWlssid9KyE',
'spotify:artist:1lMaDSraU3oiNUsVWJLHdF',
'spotify:artist:1Cs0zKBU1kc0i8ypK3B9ai',
'spotify:artist:3DvdryKH4O95ZnsUZJKXpt',
'spotify:artist:4TwHRCIu3Xg9fjS3l7owkp',
'spotify:artist:396Kh0m4wGUvcMUULw71yi',
'spotify:artist:396Kh0m4wGUvcMUULw71yi',
'spotify:artist:2hUkfSXyuDfQbUeNOShRiz',
'spotify:artist:1l8Fu6IkuTP0U5QetQJ5Xt',
'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR',
'spotify:artist:07jc1IMGtecDZzOL9cP50t',
'spotify:artist:1GwxXgEc6oxCKQ5wykWXFs',
'spotify:artist:1eEfMU2AhEo7XnKgL7c304',
'spotify:artist:3alW3LYQS8K29z8C8NSLIX',
'spotify:artist:0Sq7oGrYEe0BDmb13wgjOO',
'spotify:artist:4YPqbAiLzBg5DIfsgQZ8QK',
'spotify:artist:78udmCDNaDu5jqfVnBqwia',
'spotify:artist:319yZVtYM9MBGqmSQnMyY6',
'spotify:artist:6k0IBR0lU42s2GYpNX7kA9',
'spotify:artist:0AunegKKH2ys98aLxq8G1A',
'spotify:artist:1255GTUKNCLCTvH9ctD4cT',
'spotify:artist:08p7B5OtcUuVblvkQIlBhJ',
'spotify:artist:7Aq8lpLMSt1Zxu56pe9bmp',
'spotify:artist:05RZIdfz59ZW2FvFuwnmNK',
'spotify:artist:33C3ByGk9NCrjTIXr6npqf',
'spotify:artist:2MGT7CYlixSPxAnnkhLpaA',
'spotify:artist:4sbB4Yy6Qig51pKCIKSLw3',
'spotify:artist:5Z1reBuaReg2pg72I2mnMm',
'spotify:artist:0H0yrKDty3I7pPrNUSH3wW',
'spotify:artist:6x9QLdzo6eBZxJ1bHsDkjg',
'spotify:artist:5KNqYrivNgVCHBssEUSu5B',
'spotify:artist:6JW8wliOEwaDZ231ZY7cf4',
'spotify:artist:6irRU9KRGb0cMtlsBZMg2i',
'spotify:artist:5EvFsr3kj42KNv97ZEnqij',
'spotify:artist:49FeZO3eSrJs7oH7lYLU1r',
'spotify:artist:1urjDGTd4iBze91Z1W1gu7',
'spotify:artist:6dQgkiJm1fRkcqKOvPzysr',
'spotify:artist:5z2cHsOmmmWcAPNG1oEhw0',
'spotify:artist:2AQjGvtT0pFYfxR3neFcvz',
'spotify:artist:12d4iIvTOk7JkI6ecvc3ca',
'spotify:artist:59hLmB5DrdihCYtNeFeW1U',
'spotify:artist:77SW9BnxLY8rJ0RciFqkHh',
'spotify:artist:7slfeZO9LsJbWgpkIoXBUJ',
'spotify:artist:7dnB1wSxbYa8CejeVg98hz',
'spotify:artist:0TorzrcTerTn7l5q9LVjmt',
'spotify:artist:6tbjWDEIzxoDsBA1FuhfPW',
'spotify:artist:4JCt4xrbbBB9blkKwNlcJ7',
'spotify:artist:0PFtn5NtBbbUNbU9EAmIWF',
'spotify:artist:2gBjLmx6zQnFGQJCAQpRgw',
'spotify:artist:7352aRY2mqSxBZwzUb6LmA',
'spotify:artist:6NWtt9pNOL2Gx7kBykdE5x',
'spotify:artist:6w6OUqDUOPDvEHY85kbJRE',
'spotify:artist:0fiWOxhsBsQQvFDtxUQWo0',
'spotify:artist:3oSJ7TBVCWMDMiYjXNiCKE',
'spotify:artist:718COspgdWOnwOFpJHRZHS',
'spotify:artist:4oUHIQIBe0LHzYfvXNW4QM',
'spotify:artist:0Ty63ceoRnnJKVEYP0VQpk',
'spotify:artist:3pFCERyEiP5xeN2EsPXhjI',
'spotify:artist:4JeqRr8Upw5uxLEu6jgIRm',
'spotify:artist:4eQeZFlDU3froIAkzS6vnC',
'spotify:artist:3bcLBxvaI7GsBzGp3WHnwQ',
'spotify:artist:4xRYI6VqpkE3UwrDrAZL8L',
'spotify:artist:6TIYQ3jFPwQSRmorSezPxX',
'spotify:artist:08rMCq2ek1YjdDBsCPVH2s',
'spotify:artist:3FD9VAd0ppjmKbfxF4LzUh',
'spotify:artist:3Y3xIwWyq5wnNHPp5gPjOW',
'spotify:artist:2IBPIcSkvQZyoaVfoyZKE8',
'spotify:artist:0G7OYsWptjRzVFT1AxP8TS',
'spotify:artist:7HLvzuM9p11k9lUQfSM4Rq',
'spotify:artist:0cc6vw3VN8YlIcvr1v7tBL',
'spotify:artist:1dLWg6m8RRhizsdqJbhyj3',
'spotify:artist:4opTS86dN9uO313J9CE8xg',
'spotify:artist:4PJHDzdFoQcklrWU18QdsU',
'spotify:artist:0XKOBt59crntr7HQXXO8Yz',
'spotify:artist:0dlwjJe4kg9TgOMN184bJF',
'spotify:artist:37W8SKV5Vc8lcGtdyWCA5m',
'spotify:artist:0iEVT8dLNSSmipPOrDGu2z',
'spotify:artist:7zICaxnDB9ZprDSiFpvbbW',
'spotify:artist:5yV1qdnmxyIYiSFB02wpDj',
'spotify:artist:7H55rcKCfwqkyDFH9wpKM6',
'spotify:artist:2Sqr0DXoaYABbjBo9HaMkM',
'spotify:artist:6eoJpTIlcuxJNjV5fDzDJH',
'spotify:artist:1Fj0R2t4HaJa3oUe8azB8R',
'spotify:artist:4aP1lp10BRYZO658B2NwkG',
'spotify:artist:13ZEDW6vyBF12HYcZRr4EV',
'spotify:artist:4YLtscXsxbVgi031ovDDdh',
'spotify:artist:6Xgp2XMz1fhVYe7i6yNAax',
'spotify:artist:3f5fB1YUbkUnW7cf7I0R48',
'spotify:artist:2sgj794K3UaW1ApX28ZUhY',
'spotify:artist:0TcYeHEK9sBtv7xPbKhzHz',
'spotify:artist:1LFKKuzn302wp15dYH28id',
'spotify:artist:4ntkql3f3ect7NDRUJ7aAY',
'spotify:artist:3VJakY5Yw5phAOF4Bada5v',
'spotify:artist:08ct2eZF5lUPdJpHwNKWof',
'spotify:artist:5SHQUMAmEK5KmuSb0aDvsn',
'spotify:artist:0OtigLoCgLYvXnMxbE5qH5',
'spotify:artist:67RWyN1fDOu7WuSTIi5hE7',
'spotify:artist:7x8nK0m0cP2ksQf0mjWdPS',
'spotify:artist:4BYxqVkZyFjtik7crYLg5Q',
'spotify:artist:0oBEeN6BCxEgMogzThqrPf',
'spotify:artist:4aJTB79uwT4sP0Nb8QGWHc',
'spotify:artist:1zhIzONVRlTfD1VQRauHMl',
'spotify:artist:3grHWM9bx2E9vwJCdlRv9O',
'spotify:artist:7FY5V3XMwlNBPitEjXowHQ',
'spotify:artist:776Uo845nYHJpNaStv1Ds4',
'spotify:artist:6GI52t8N5F02MxU0g5U69P',
'spotify:artist:5nqr3PugZYyBcQrnKtFkAu',
'spotify:artist:1KfBKoozFugeT1PBkz33xl',
'spotify:artist:1CYsQCypByMVgnv17qsSbQ',
'spotify:artist:5lkiCO9UQ8B23dZ1o0UV4m',
'spotify:artist:6v8FB84lnmJs434UJf2Mrm',
'spotify:artist:2pdvghEHZJtgSXZ7cvNLou',
'spotify:artist:6B5c4sch27tWHAGdarpPaW',
'spotify:artist:4L1z1IcfK7lbqx8izGHaw5',
'spotify:artist:7d7q5Y1p2QWS4QRAhTQR5E',
'spotify:artist:5lpH0xAS4fVfLkACg9DAuM',
'spotify:artist:0gxyHStUsqpMadRV0Di1Qt',
'spotify:artist:137W8MRPWKqSmrBGDBFSop',
'spotify:artist:1ThoqLcyIYvZn7iWbj8fsj',
'spotify:artist:3dkbV4qihUeMsqN4vBGg93',
'spotify:artist:7GaxyUddsPok8BuhxN6OUW',
'spotify:artist:3IKV7o6WPphDB7cCWXaG3E',
'spotify:artist:7ukOC5U2oweIhxAm26BNF7',
'spotify:artist:0smy8yDrRoI4CnhpOuthg0',
'spotify:artist:5PN2aHIvLEM98XIorsPMhE',
'spotify:artist:0kObWap02DEg9EAJ3PBxzf',
'spotify:artist:1zxDewzd2j1ZdSBGaYcr0y',
'spotify:artist:7Bah8E0kCETqEpAHI6CPzQ',
'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR',
'spotify:artist:0eGh2jSWPBX5GuqIHoZJZG',
'spotify:artist:7jVv8c5Fj3E9VhNjxT4snq',
'spotify:artist:5fhMbh4PVSLSODF2fhWwqt',
'spotify:artist:7mnBLXK823vNxN3UWB7Gfz',
'spotify:artist:6kEsKkXuE4olX7m3mNLZkS',
'spotify:artist:2LB9H0px4qWbCHg9Axzhga',
'spotify:artist:5SbkVQYYzlw1kte75QIabH',
'spotify:artist:4Tvos0a5rRrBu4Oodu5f79',
'spotify:artist:60rpJ9SgigSd16DOAG7GSa',
'spotify:artist:7y9COUDxusQXRjW95vOubE',
'spotify:artist:2TI7qyDE0QfyOlnbtfDo7L',
'spotify:artist:7rvkQK7ZqO34sKsWEXjz3R',
'spotify:artist:2dyeCWctcFRt3Pha76ONgb',
'spotify:artist:2dyeCWctcFRt3Pha76ONgb',
'spotify:artist:7wCjDgV6nqBsHguQXPAaIM',
'spotify:artist:07VmOvmuBp9G0gb8BTrpn0',
'spotify:artist:6I3M904Y9IwgDjrQ9pANiB',
'spotify:artist:1Be36RHAlqJpfUt3tsmUQD',
'spotify:artist:2mDlls63YpRsMjgpyXxyu6',
'spotify:artist:4WlSvDKaq1PA2Nr7cCIPxX',
'spotify:artist:0KzlJXg6A3NVSCkRjecKnS',
'spotify:artist:0k17h0D3J5VfsdmQ1iZtE9',
'spotify:artist:7CajNmpbOovFoOoasH2HaY',
'spotify:artist:59B2CHEMh1to8diq0jB92r',
'spotify:artist:3kTcyaSYP5fqlqrNUvxiER',
'spotify:artist:7H6dkUChT5EoOQtUVMg4cN',
'spotify:artist:05YVYeV4HxYp5rrWalvuE1',
'spotify:artist:2MMaIlHd8UvDW0mVRAncD6',
'spotify:artist:0AVE7rDx4X9w0pW1XlN1om',
'spotify:artist:7d8R11w4QG2ihJeTaZq37F',
'spotify:artist:51d3rUlLuMh7EW09aMe7yj',
'spotify:artist:3R6c2tWDBeYB1GncjLMn9n',
'spotify:artist:78SHxLdtysAXgywQ4vE0Oa',
'spotify:artist:545idYhdloaSlLGBY8E9u2',
'spotify:artist:1SZMaiNHfdUuU0qZKZ6y62',
'spotify:artist:4VMYDCV2IEDYJArk749S6m',
'spotify:artist:1iNqsUDUraNWrj00bqssQG',
'spotify:artist:1VDurPMbOKrNU8mcMP4X1P',
'spotify:artist:4C3uGP8vRDzxrhJxZiOjTe',
'spotify:artist:5dcOK4stT4JDkP6Dqhbz5s',
'spotify:artist:1yMYjh77WgOVafRkI50mim',
'spotify:artist:3X458ddYA2YcVWuVIGGOYe',
'spotify:artist:3mIj9lX2MWuHmhNCA7LSCW',
'spotify:artist:1fBCIkoPOPCDLUxGuWNvyo',
'spotify:artist:5P5FTygHyx2G57oszR3Wot',
'spotify:artist:6BMhCQJYHxxKAeqYS1p5rY',
'spotify:artist:0EyuKHE1AeE9lWUF8mzKVp',
'spotify:artist:74eX4C98E4FCrAMl39qRsJ',
'spotify:artist:6PXS4YHDkKvl1wkIl4V8DL',
'spotify:artist:0AuW7OCyKfFrsMbtHrYgIV',
'spotify:artist:44PA0rCQXikgOWbfY7Fq7m',
'spotify:artist:06HL4z0CvFAxyc27GXpf02',
'spotify:artist:5GUVj2b1lJ4DolQyHlzyaO',
'spotify:artist:6PAt558ZEZl0DmdXlnjMgD',
'spotify:artist:7iZtZyCzp3LItcw1wtPI3D',
'spotify:artist:2x9SpqnPi8rlE9pjHBwmSC',
'spotify:artist:7mPcfx7PhnBh6n4HMCd61A',
'spotify:artist:2GaayiIs1kcyNqRXQuzp35',
'spotify:artist:5Va9LuEmaZxnbk1gMnjMD7',
'spotify:artist:3CQIn7N5CuRDP8wEI7FiDA',
'spotify:artist:2tRsMl4eGxwoNabM08Dm4I',
'spotify:artist:6KWcxMWVNVIYbdOQyJtsSy',
'spotify:artist:242iqFnwNhlidVBMI9GYKp',
'spotify:artist:2GqgiChw79vIYobk2zJFsO',
'spotify:artist:2q0HDEYPYC2oZjXgYqAWMZ',
'spotify:artist:17lzZA2AlOHwCwFALHttmp',
'spotify:artist:1sBkRIssrMs1AbVkOJbc7a',
'spotify:artist:4yBK75WVCQXej1p04GWqxH',
'spotify:artist:6VuMaDnrHyPL1p4EHjYLi7',
'spotify:artist:218kRJZ7FJs0hWIk8Ynzhz',
'spotify:artist:3vMWY1bUrmYoSoEWLiQWOZ',
'spotify:artist:3lPQ2Fk5JOwGWAF3ORFCqH',
'spotify:artist:6zFYqv1mOsgBRQbae3JJ9e',
'spotify:artist:7CwJ4VEWs7HV7MWQt9cgyk',
'spotify:artist:5gxynDEKwNDgxGJmJjZyte',
'spotify:artist:6P7H3ai06vU1sGvdpBwDmE',
'spotify:artist:3GBPw9NK25X1Wt2OUvOwY3',
'spotify:artist:3GBPw9NK25X1Wt2OUvOwY3',
'spotify:artist:5fU6lODhpw3GEGGJuaDprR',
'spotify:artist:5ZEAzHE2TzAwUcOj6jMIgf',
'spotify:artist:502FY8XQYRxClWgolQ3Hf2',
'spotify:artist:3cMyqwpNT2D8f0WuaRzzZX',
'spotify:artist:1g8uGVofjr2ApBrI8OZhJf',
'spotify:artist:4qBgvVog0wzW75IQ48mU7v',
'spotify:artist:3TOqt5oJwL9BE2NG9MEwDa',
'spotify:artist:7bvcQXJHkFiN1ppIN3q4fi',
'spotify:artist:3F2Y683ovlWT53WKCeq0oj',
'spotify:artist:2jzc5TC5TVFLXQlBNiIUzE',
'spotify:artist:2kucQ9jQwuD8jWdtR9Ef38',
'spotify:artist:1mfDfLsMxYcOOZkzBxvSVW',
'spotify:artist:360IAlyVv4PCEVjgyMZrxK',
'spotify:artist:73sIBHcqh3Z3NyqHKZ7FOL',
'spotify:artist:2h93pZq0e7k5yf4dywlkpM',
'spotify:artist:4V8LLVI7PbaPR0K2TGSxFF',
'spotify:artist:4LLpKhyESsyAXpc4laK94U',
'spotify:artist:13ubrt8QOOCPljQ2FL1Kca',
'spotify:artist:0jgAONnsHxrwAlhkMUVS78',
'spotify:artist:5iB5AWIa7qreioi0AF3Bxa',
'spotify:artist:4kNRotUiNCcfZhnpJJTDtl',
'spotify:artist:4Ti0EKl2PVEms2NRMVGqNe',
'spotify:artist:3XnO697XIus1M0cMuxZjos',
'spotify:artist:55yEsJBqyn27JPsw5hHzqi',
'spotify:artist:4uaaRGr9KMfYPDaUnBIngc',
'spotify:artist:6Sb8EHpZvyjSTT5KD9DjUZ',
'spotify:artist:4EnEZVjo3w1cwcQYePccay',
'spotify:artist:3Mcii5XWf6E0lrY3Uky4cA',
'spotify:artist:7bu3H8JO7d0UbMoVzbo70s',
'spotify:artist:3iTsJGG39nMg9YiolUgLMQ',
'spotify:artist:4tw2Lmn9tTPUv7Gy7mVPI4',
'spotify:artist:4awU3JRQXIYoxabdwO92AU',
'spotify:artist:0MBIKH9DjtBkv8O3nS6szj',
'spotify:artist:7FgMLbnZVrEnir95O0YujA',
'spotify:artist:6AcOTCYBMvjKYy4zms0kaC',
'spotify:artist:0yNLKJebCb8Aueb54LYya3',
'spotify:artist:6WY7D3jk8zTrHtmkqqo5GI',
'spotify:artist:19eLuQmk9aCobbVDHc6eek',
'spotify:artist:5WUlDfRSoLAfcVSX1WnrxN',
'spotify:artist:7Hv1GlNkBy4muBAWYOlfpz',
'spotify:artist:2BGRfQgtzikz1pzAD0kaEn',
'spotify:artist:7AC976RDJzL2asmZuz7qil',
'spotify:artist:6w6z8m4WXX7Tub4Rb6Lu7R',
'spotify:artist:0roeI3yPusDWwWRzAqTopw',
'spotify:artist:5TDVKqW9uhqGjwwwKGuma4',
'spotify:artist:0uq5PttqEjj3IH1bzwcrXF',
'spotify:artist:3FjOdEflyH6wWrmTfj4xUo',
'spotify:artist:0DLAUCjsXu9E1Pj28NTZZq',
'spotify:artist:4qwGe91Bz9K2T8jXTZ815W',
'spotify:artist:0H4sfMNK9hoasHItAgD1kJ',
'spotify:artist:1T0wRBO0CK0vK8ouUMqEl5',
'spotify:artist:6umNMJq7B8UdOVmwPV6PRL',
'spotify:artist:2ycnb8Er79LoH2AsR5ldjh',
'spotify:artist:6Mo9PoU6svvhgEum7wh2Nd',
'spotify:artist:1W9qOBYRTfP7HcizWN43G1',
'spotify:artist:2Mu5NfyYm8n5iTomuKAEHl',
'spotify:artist:0CEFCo8288kQU7mJi25s6E',
'spotify:artist:73ZPfpfg1LBVvDEArK4l5B',
'spotify:artist:0ZcJXldoq09BRIMl0Qh1Vm',
'spotify:artist:1sBRcMH8DDR8Nsk2RoJmjS',
'spotify:artist:2DlGxzQSjYe5N6G9nkYghR',
'spotify:artist:6TcnmlCSxihzWOQJ8k0rNS',
'spotify:artist:3REpOYo13YkVj1dFzda12A',
'spotify:artist:0vEsuISMWAKNctLlUAhSZC',
'spotify:artist:4Eqd24yS5YcxI8b6Xfuwr8',
'spotify:artist:7AQzXO3NPNQsI7oNu5rC3r',
'spotify:artist:3AHq6rutf72JF0ul8GB6G2',
'spotify:artist:3vRclCt9VnNhYIxFMQCxuM',
'spotify:artist:71aKjsWKYqASAffyIQaocZ',
'spotify:artist:1vgjN6nIPNKiiQGE9PtzTT',
'spotify:artist:1vgjN6nIPNKiiQGE9PtzTT',
'spotify:artist:32lVGr0fSRGT6okLKHiP68',
'spotify:artist:6V7pNWhlJpD0s0bMdB1PU9',
'spotify:artist:5OfhOoKunSnuubxxRML8J3',
'spotify:artist:5MmXZQUGs0eXP5PV79iKrB',
'spotify:artist:3ScY9CQxNLQei8Umvpx5g6',
'spotify:artist:1J2VVASYAamtQ3Bt8wGgA6',
'spotify:artist:1XkoF8ryArs86LZvFOkbyr',
'spotify:artist:20JZFwl6HVl6yg8a4H3ZqK',
'spotify:artist:3nJWBJvK7uGvfp4iZh9CkN',
'spotify:artist:1TqQi97nqeiuOJrIFv5Sw0',
'spotify:artist:0dEvJpkqhrcn64d3oI8v79',
'spotify:artist:22TEmHXBBLjTec2LOAuMdS',
'spotify:artist:2OgUPVlWYgGBGMefZgGvCO',
'spotify:artist:1yxSLGMDHlW21z4YXirZDS',
'spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ',
'spotify:artist:5QsUbpxSE8lCZ5ga5rnD22',
'spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx',
'spotify:artist:26VFTg2z8YR0cCuwLzESi2',
'spotify:artist:64M6ah0SkkRsnPGtGiRAbb',
'spotify:artist:2wUjUUtkb5lvLKcGKsKqsR',
'spotify:artist:6JL8zeS1NmiOftqZTRgdTz',
'spotify:artist:6jJ0s89eD6GaHleKKya26X',
'spotify:artist:530Sdm7eqqzWBdDmILMgnu',
'spotify:artist:6clbbhnIqpHnqxwtOWcilg',
'spotify:artist:6jdObwsrIjSRnBbMw6lPBj',
'spotify:artist:7Js6Lde8thlIHXggv2SCBz',
'spotify:artist:0L5fC7Ogm2YwgqVCRcF1bT',
'spotify:artist:3aBkeBhwadnWMWoVJ2CxJC',
'spotify:artist:5BvJzeQpmsdsFp4HGUYUEx',
'spotify:artist:7rkW85dBwwrJtlHRDkJDAC',
'spotify:artist:0TnOYISbd1XYRBk9myaseg',
'spotify:artist:6AVLthptCPhfrxlHadOBJD',
'spotify:artist:2DzRMyWgjuMbYvt5BLbpCo',
'spotify:artist:7Gf3LSwa5hh8Cjo60WhVjC',
'spotify:artist:6I6t21SFbxZ7RbQgD5dN7U',
'spotify:artist:1f2e3RQf7LHOum8NU61q0R',
'spotify:artist:2hWr3AjjKOCVmWcwvuT4uM',
'spotify:artist:6x2LnllRG5uGarZMsD4iO8',
'spotify:artist:3c8BXo73tAg7XZiNHJugZm',
'spotify:artist:5eKLa1xyHLq8ERWmT1CRHj',
'spotify:artist:4n31svBA9GGIYxGxgrQaRK',
'spotify:artist:5q4AzEtCoYJyXjMMoEkSU5',
'spotify:artist:5hfOHpGMsi8O3DA0ROLGqF',
'spotify:artist:5AEG63ajney2BoDXi0Vb84',
'spotify:artist:0L8ExT028jH3ddEcZwqJJ5',
'spotify:artist:0rvjqX7ttXeg3mTy8Xscbt',
'spotify:artist:0gadJ2b9A4SKsB1RFkBb66',
'spotify:artist:7LfRnzGM0wRSyS5mmAwJbv',
'spotify:artist:7FBcuc1gsnv6Y1nwFtNRCb',
'spotify:artist:0qSX3s5pJnAlSsgsCne8Cz',
'spotify:artist:6mcrZQmgzFGRWf7C0SObou',
'spotify:artist:7qeBsFLeNEuzrDhDMbnDwF',
'spotify:artist:4LtIfuFeAamus9OOItuWDD',
'spotify:artist:2ZvrvbQNrHKwjT7qfGFFUW',
'spotify:artist:5xLSa7l4IV1gsQfhAMvl0U',
'spotify:artist:1dfeR4HaWDbWqFHLkxsg1d',
'spotify:artist:1lOQRP0UJWmpoDWBa8PB0B',
'spotify:artist:3hcs9uc56yIGFCSy9leWe7',
'spotify:artist:2CxLP749mup3ncPrXgCnvU',
'spotify:artist:7aBzpmFXB4WWpPl2F7RjBe',
'spotify:artist:1c22GXH30ijlOfXhfLz9Df',
'spotify:artist:0KDuKk6YdEu3hR56HtXmxt',
'spotify:artist:1QPZhx0asYTBxD5RDqfU49',
'spotify:artist:1ii6e2pv8VIRwnTER71rMl',
'spotify:artist:6AnrSlk5Gp1YMXgaI3mWCL',
'spotify:artist:1tqZaCwM57UFKjWoYwMLrw',
'spotify:artist:0cQbJU1aAzvbEmTuljWLlF',
'spotify:artist:3HC7NcxQx2Yk0fWoRKJ0xF',
'spotify:artist:0reZZVbAPxgX1Rqj6XbWj3',
'spotify:artist:0hEurMDQu99nJRq8pTxO14',
'spotify:artist:7C4sUpWGlTy7IANjruj02I',
'spotify:artist:0nCiidE5GgDrc5kWN3NZgZ',
'spotify:artist:3YR92OLKlvkK5oKNekSqXe',
'spotify:artist:0zokIKbH4c7crvZP9Xel1Y',
'spotify:artist:5SMVzTJyKFJ7TUb46DglcH',
'spotify:artist:3Bhgbs9kIwl8SuU0eIqN6b',
'spotify:artist:7ctAOUlIAs7yuMODWE2Fyz',
'spotify:artist:3sFhA6G1N0gG1pszb6kk1m',
'spotify:artist:4x1nvY2FN8jxqAFA0DA02H',
'spotify:artist:6DbJi8AcN5ANdtvJcwBSw8',
'spotify:artist:7gi3jmwpUpNWdswT8eEprF',
'spotify:artist:7gi3jmwpUpNWdswT8eEprF',
'spotify:artist:2ptmyXoL7poH6Zq62h1QT9',
'spotify:artist:0RYyGExpy57GLGa1GtFoVo',
'spotify:artist:7vNcTk9TgKF0qDsS87nWGE',
'spotify:artist:41F8cUV8LJGVDPNnj0ito3',
'spotify:artist:0DW7boyjvbaSP3OJ72sXgC',
'spotify:artist:2yQf6b8hxahZaT5dHlWaB1',
'spotify:artist:64rxQRJsLgZwHHyWKB8fiF',
'spotify:artist:04LIHk1SobiQwt2tlupoAV',
'spotify:artist:4j7EVY3kuDwLPfD2jfC7LC',
'spotify:artist:2PSiyldxmJze7xiqbz658m',
'spotify:artist:7FukyKMUPL10mDN9LgAngI',
'spotify:artist:3MdG05syQeRYPPcClLaUGl',
'spotify:artist:5dbuFbrHa1SJlQhQX9OUJ2',
'spotify:artist:0dCKce6tJJdHvlWnDMwzPW',
'spotify:artist:2eogQKWWoohI3BSnoG7E2U',
'spotify:artist:1G9G7WwrXka3Z1r7aIDjI7',
'spotify:artist:2cFrymmkijnjDg9SS92EPM',
'spotify:artist:7EK1bQADBoqbYXnT4Cqv9w',
'spotify:artist:7EK1bQADBoqbYXnT4Cqv9w',
'spotify:artist:5krkohEVJYw0qoB5VWwxaC',
'spotify:artist:50NoVNy9GU1lCrDV8iGpyu',
'spotify:artist:5daXMUbrPh8oX9Nd4CnToG',
'spotify:artist:49mU7Sl98U8MkECsd6zVq0',
'spotify:artist:6nDLku5uL3ou60kvCGZorh',
'spotify:artist:6hxlIANxRjG8mQTcR8evvP',
'spotify:artist:3q7HBObVc0L8jNeTe5Gofh',
'spotify:artist:5f7VJjfbwm532GiveGC0ZK',
'spotify:artist:44gRHbEm4Uqa0ykW0rDTNk',
'spotify:artist:31TPClRtHm23RisEBtV3X7',
'spotify:artist:26dSoYclwsYLMAKD3tpOr4',
'spotify:artist:6FD0unjzGQhX3b6eMccMJe',
'spotify:artist:57LYzLEk2LcFghVwuWbcuS',
'spotify:artist:2Ev0e8GUIX4u7d9etNLTXg',
'spotify:artist:7gOdHgIoIKoe4i9Tta6qdD',
'spotify:artist:5YGY8feqx7naU7z4HrwZM6',
'spotify:artist:4ogvuDRerGhZfSf7TtzHlr',
'spotify:artist:3rfgbfpPSfXY40lzRK7Syt',
'spotify:artist:23rleGXVOVVgTk3xgtmfE4',
'spotify:artist:1R6Hx1tJ2VOUyodEpC12xM',
'spotify:artist:1GxkXlMwML1oSg5eLPiAz3',
'spotify:artist:2UBTfUoLI07iRqGeUrwhZh',
'spotify:artist:7Ead768rc4ShGxnqtqccU5',
'spotify:artist:3P9jdZehQvmAoTrWU1p5zt',
'spotify:artist:74XFHRwlV6OrjEM0A2NCMF',
'spotify:artist:7j9PMegEgVN1fNp8NZXNCI',
'spotify:artist:7j9PMegEgVN1fNp8NZXNCI',
'spotify:artist:6IRouO5mvvfcyxtPDKMYFN',
'spotify:artist:43mhFhQ4JAknA7Ik1bOZuV',
'spotify:artist:16GcWuvvybAoaHr0NqT8Eh',
'spotify:artist:1OxJzMLmR9l5zPLap9OxuO',
'spotify:artist:6h3rSZ8VLK7a5vXjEmhfuD',
'spotify:artist:4KHdmkq99PXA6QEJ2lKpA3',
'spotify:artist:7BEfMxbaqx6dOpbtlEqScm',
'spotify:artist:6ZzOIAEw7joAghyA80c6D0',
'spotify:artist:09L3cUdx0hq6qn5bKuJJ4I',
'spotify:artist:09L3cUdx0hq6qn5bKuJJ4I',
'spotify:artist:35y7CZMg7jbG8Q96JY7dyC',
'spotify:artist:2HBbky0Z08ZcCKVsXWbNE4',
'spotify:artist:71lGEtP9qYXDsSXjfexTqO',
'spotify:artist:4vRSocKbGh7PsQrYRDVMEF',
'spotify:artist:3OsUvoKmZA2gw8rETtxGlz',
'spotify:artist:6S0GHTqz5sxK5f9HtLXn9q',
'spotify:artist:0MMNJf1JrxeSFE2pcpX50b',
'spotify:artist:2F0kPpQ5mtta1ORIyO2xex',
'spotify:artist:4lOXDaGBU2mZM3PZU35ar0',
'spotify:artist:26AHtbjWKiwYzsoGoUZq53',
'spotify:artist:4RddZ3iHvSpGV4dvATac9X',
'spotify:artist:02kJSzxNuaWGqwubyUba0Z',
'spotify:artist:37w38cCSGgKLdayTRjna4W',
'spotify:artist:4iSKnRZAxkmqNok6tv10Se',
'spotify:artist:4P0dddbxPil35MNN9G2MEX',
'spotify:artist:0X380XXQSNBYuleKzav5UO',
'spotify:artist:5jVeqi3PNaTOajfvBa4uFn',
'spotify:artist:4bthk9UfsYUYdcFyqxmSUU',
'spotify:artist:3lJ6jvOiNY46L2juqXufpf',
'spotify:artist:5aEWnrN8h3MhuFUPRfaVuy',
'spotify:artist:772SIFJQiXTCfxncTK1UMn',
'spotify:artist:77tT1kLj6mCWtFNqiOmP9H',
'spotify:artist:77tT1kLj6mCWtFNqiOmP9H',
'spotify:artist:25u4wHJWxCA9vO0CzxAbK7',
'spotify:artist:72KyoXzp0NOQij6OcmZUxk',
'spotify:artist:11kBu957KTYoAltZHDm8gW',
'spotify:artist:488v7rQzthLNK22r0UvMie',
'spotify:artist:1eClJfHLoDI4rZe5HxzBFv',
'spotify:artist:3zZ88AwlTwfCJkowsFCvLA',
'spotify:artist:6cXMpsP9x0SH4kFfMyVezF',
'spotify:artist:3EA9hVIzKfFiQI0Kikz2wo',
'spotify:artist:6J7biCazzYhU3gM9j1wfid',
'spotify:artist:69MEO1AADKg1IZrq2XLzo5',
'spotify:artist:18YelxQioKKPXK1PE5hh1n',
'spotify:artist:3kjuyTCjPG1WMFCiyc5IuB',
'spotify:artist:2kreKea2n96dXjcyAU9j5N',
'spotify:artist:1WRM9i067hd2ujxxi8FI3m',
'spotify:artist:2AV6XDIs32ofIJhkkDevjm',
'spotify:artist:6UL7BodGc5iVmQGlMwHR0g',
'spotify:artist:6XwcepfAsPhrvwziGxhwju',
'spotify:artist:1W3FSF1BLpY3hlVIgvenLz',
'spotify:artist:1W3FSF1BLpY3hlVIgvenLz',
'spotify:artist:20s0P9QLxGqKuCsGwFsp7w',
'spotify:artist:1qAuetfG6mhtDgsVIffWQc',
'spotify:artist:44NX2ffIYHr6D4n7RaZF7A',
'spotify:artist:28AyklUmMECPwdfo8NEsV0',
'spotify:artist:4MoAOfV4ROWofLG3a3hhBN',
'spotify:artist:2IvkS5MXK0vPGnwyJsrEyV',
'spotify:artist:5Zq7R5qmi58ByYyBQTlNuk',
'spotify:artist:3FfvYsEGaIb52QPXhg4DcH',
'spotify:artist:6yJCxee7QumYr820xdIsjo',
'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V',
'spotify:artist:4SQxI8xg6LcSWL3KuGVHLa',
'spotify:artist:0qEcf3SFlpRcb3lK3f2GZI',
'spotify:artist:49iKbKGqgn8OESkW5WduX0',
'spotify:artist:7KMqksf0UMdyA0UCf4R3ux',
'spotify:artist:4phGZZrJZRo4ElhRtViYdl',
'spotify:artist:4EzkuveR9pLvDVFNx6foYD',
'spotify:artist:7F7zi7OcJYBOmSFZIYYFCR',
'spotify:artist:6OqQbeNe7Pi7Zi7SNTi9ko',
'spotify:artist:7qvsLYsYP0MHD7jkdv6DAG',
'spotify:artist:2uB5sjjJwj4kB6TrRGLYJO',
'spotify:artist:2qFe0FyUMK8XXoyOsfYJr2',
'spotify:artist:3dvIPsJH7o9xnBwFxowzof',
'spotify:artist:40Yq4vzPs9VNUrIBG5Jr2i',
'spotify:artist:3iuobQyYhofKRr5W6HZoKt',
'spotify:artist:5yzE49FicYiSxN61oaxkNn',
'spotify:artist:6ktyNEnEukKzskQEiXgCFD',
'spotify:artist:01QTIT5P1pFP3QnnFSdsJf',
'spotify:artist:0EeQBlQJFiAfJeVN2vT9s0',
'spotify:artist:4P70aqttdpJ9vuYFDmf7f6',
'spotify:artist:3d2hJTVTwo08F9b0ZFQukJ',
'spotify:artist:3PcMolaGsh2nryvBR2sQOS',
'spotify:artist:39T6qqI0jDtSWWioX8eGJz',
'spotify:artist:3DiDSECUqqY1AuBP8qtaIa',
'spotify:artist:2LJxr7Pt3JnP60eLxwbDOu',
'spotify:artist:2EMAnMvWE2eb56ToJVfCWs',
'spotify:artist:6k7z2h1jKogQyArcLI13wy',
'spotify:artist:0NyzfcGDZZ6GM25EBG9BYK',
'spotify:artist:75FnCoo4FBxH5K1Rrx0k5A',
'spotify:artist:4TMHGUX5WI7OOm53PqSDAT',
'spotify:artist:43ZHCT0cAZBISjO8DG9PnE',
'spotify:artist:0auu2itHTxEdAMRHvx7CyG',
'spotify:artist:5cMgGlA1xGyeAB2ctYlRdZ',
'spotify:artist:3PyWEKLWI0vHPmoNrIX0QE',
'spotify:artist:0KV1YtheX8zbH3rBACpKL3',
'spotify:artist:1l7ZsJRRS8wlW3WfJfPfNS',
'spotify:artist:1KCSPY1glIKqW2TotWuXOR',
'spotify:artist:39oSLGo3HkaeYXzUEGgAGQ',
'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX',
'spotify:artist:7IfculRW2WXyzNQ8djX8WX',
'spotify:artist:0vn7UBvSQECKJm2817Yf1P',
'spotify:artist:42q4Ivs7tAiCZ5C7eG5q4c',
'spotify:artist:4afEuMmqeS7G5nlKF3jSM0',
'spotify:artist:6vXTefBL93Dj5IqAWq6OTv',
'spotify:artist:22WZ7M8sxp5THdruNY3gXt',
'spotify:artist:22bE4uQ6baNwSHPVcDxLCe',
'spotify:artist:36QJpDe2go2KgaRleHCDTp',
'spotify:artist:2mwN5WQ1mAhOibFEbfFfsG',
'spotify:artist:6zmLTiN3NBNCDECRbbJKkT',
'spotify:artist:6u17YlWtW4oqFF5Hn9UU79',
'spotify:artist:1OwarW4LEHnoep20ixRA0y',
'spotify:artist:1d6dwipPrsFSJVmFTTdFSS',
'spotify:artist:738OS3zrCO782uDiUN9pet',
'spotify:artist:0dYkMe3wK29DulSa0uR8Rq',
'spotify:artist:5hW4L92KnC6dX9t7tYM4Ve',
'spotify:artist:73JEBdDEFeVaOLg3y0HhBD',
'spotify:artist:01Ppu7N8uYJI8SAONo2YZA',
'spotify:artist:72y3ZI95ctkQC2O4mjBaU3',
'spotify:artist:2NmK5FyrQ18HOPXq1UBzqa',
'spotify:artist:2r09Inibex3C4ZNTUVSG3m',
'spotify:artist:1riHqX633Kup3mJAw8WR8p',
'spotify:artist:6Jrj26oAY96EEC2lqC6fua',
'spotify:artist:0GByy3DcfbQwDvXGCWmzv9',
'spotify:artist:4N8BwYTEC6XqykGvXXlmfv',
'spotify:artist:0543y7yrvny4KymoaneT4W',
'spotify:artist:69Mj3u4FTUrpyeGNSIaU6F',
'spotify:artist:485uL27bPomh29R4JmQehQ',
'spotify:artist:6QtGlUje9TIkLrgPZrESuk',
'spotify:artist:1pdGj6Gak10Q2ajig4l1Dm',
'spotify:artist:1co4F2pPNH8JjTutZkmgSm',
'spotify:artist:3Wsbuy58Ku5N0hWjTuERRa',
'spotify:artist:1iBpJEBrSFXCtPOc5OUZiY',
'spotify:artist:3mY9Ii0cL5SQxpOTAm8SHx',
'spotify:artist:4KDyYWR7IpxZ7xrdYbKrqY',
'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2',
'spotify:artist:4STHEaNw4mPZ2tzheohgXB',
'spotify:artist:6zB02lwP6L6ZH32nggQiJT',
'spotify:artist:0EeHVtSdrYibpGDVHjWEpe',
'spotify:artist:6IFXsrXBpwbIqtOUOiAa3p',
'spotify:artist:6nwh8AhPpbwOUnHZJWrHT6',
'spotify:artist:5fsDcuclIe8ZiBD5P787K1',
'spotify:artist:5fsDcuclIe8ZiBD5P787K1',
'spotify:artist:5fsDcuclIe8ZiBD5P787K1',
'spotify:artist:7Ey4PD4MYsKc5I2dolUwbH',
'spotify:artist:58lV9VcRSjABbAbfWS6skp',
'spotify:artist:1bDWGdIC2hardyt55nlQgG',
'spotify:artist:77HU1Zb1VDIFvWKteJii0E',
'spotify:artist:50DI0MpQBOtBO0PFtKVQRx',
'spotify:artist:0QF3SelvOph6yRhKH8jRxU',
'spotify:artist:4aSEmLLxLX9wR5aLMllOKj',
'spotify:artist:40LHVA5BTQp9RxHOQ9JPYj',
'spotify:artist:55qiaow2sDYtjqu1mwRua6',
'spotify:artist:0Y8KmFkKOgJybpVobn1onU',
'spotify:artist:4pU3BpenOZFEBzORx2YBJW',
'spotify:artist:2AsusXITU8P25dlRNhcAbG',
'spotify:artist:13YmWQJFwgZrd4bf5IjMY4',
'spotify:artist:1By9QBFnjZAoI83BZppHlt',
'spotify:artist:2qgvmDsc4jrqbAbjq1HGM2',
'spotify:artist:1McMsnEElThX1knmY4oliG',
'spotify:artist:6Q192DXotxtaysaqNPy5yR',
'spotify:artist:29WzbAQtDnBJF09es0uddn',
'spotify:artist:4ylR3zwA0zaapAu94fktwa',
'spotify:artist:6iyGhG9V0QqRfHH25jPdZv',
'spotify:artist:6ZLTlhejhndI4Rh53vYhrY',
'spotify:artist:6ZLTlhejhndI4Rh53vYhrY',
'spotify:artist:4CYeVo5iZbtYGBN4Isc3n6',
'spotify:artist:5M52tdBnJaKSvOpJGz8mfZ',
'spotify:artist:6mdiAmATAx73kdxrNrnlao',
'spotify:artist:5c9uFWpZY2MTlk7Rft0tgp',
'spotify:artist:1EevBGfUh3RSQSGpluxgBm',
'spotify:artist:0YWxKQj2Go9CGHCp77UOyy',
'spotify:artist:3vDUJHQtqT3jFRZ2ECXDTi',
'spotify:artist:2LXeJdQVcFkhkWydphLI74',
'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi',
'spotify:artist:13aBWT8LN4oNWWGFOu96MA',
'spotify:artist:5q8HGNo0BjLWaTAhRtbwxa',
'spotify:artist:2hJPr4lk7Q8SSvCVBl9fWM',
'spotify:artist:0XNa1vTidXlvJ2gHSsRi4A',
'spotify:artist:7tYKF4w9nC0nq9CsPZTHyP',
'spotify:artist:5cj0lLjcoR7YOSnhnX0Po5',
'spotify:artist:0YinUQ50QDB7ZxSCLyQ40k',
'spotify:artist:7z5WFjZAIYejWy0NI5lv4T',
'spotify:artist:1bdytLV3FPjyhfrb6BhMej',
'spotify:artist:75L9s8KVrhCNtBUkZFnDFW',
'spotify:artist:181bsRPaVXVlUKXrxwZfHK',
'spotify:artist:3CYSRCHfilgR8DSbkCMp5j',
'spotify:artist:2Kx7MNY7cI1ENniW7vT30N',
'spotify:artist:6fNmOWQzfOVyHLQ2UqUQew',
'spotify:artist:1HY2Jd0NmPuamShAr6KMms',
'spotify:artist:4gzpq5DPGxSnKTe4SA8HAU',
'spotify:artist:1q1LI3FTPjXilhHPUH23Ax',
'spotify:artist:3d2pb1dHTm8b61zAGVUVvO',
'spotify:artist:0a1gHP0HAqALbEyxaD5Ngn',
'spotify:artist:5xuNBZoM7z1Vv8IQ6uM0p6',
'spotify:artist:2M3vnW1p5w4uPRkLYTbvdB',
'spotify:artist:4xFUf1FHVy696Q1JQZMTRj',
'spotify:artist:5NDMothbpdpq2xHqSjrrWn',
'spotify:artist:2NdeV5rLm47xAvogXrYhJX',
'spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i',
'spotify:artist:5rX1EodZfwxmW4fQX2Caot',
'spotify:artist:5Y5TRrQiqgUO4S36tzjIRZ',
'spotify:artist:5JZ7CnR6gTvEMKX4g70Amv',
'spotify:artist:2NzWRNaaLByHyI3wpuec8P',
'spotify:artist:5rkVyNGXEgeUqKkB5ccK83',
'spotify:artist:4GvEc3ANtPPjt1ZJllr5Zl',
'spotify:artist:7HkdQ0gt53LP4zmHsL0nap',
'spotify:artist:0cGUm45nv7Z6M6qdXYQGTX',
'spotify:artist:2jku7tDXc6XoB6MO2hFuqg',
'spotify:artist:5LHRHt1k9lMyONurDHEdrp',
'spotify:artist:7c0XG5cIJTrrAgEC3ULPiq',
'spotify:artist:1JHzh1ETQTMoFb2CgncnTL',
'spotify:artist:0C0XlULifJtAgn6ZNCW2eu',
'spotify:artist:3mYFuz6iY7D9X78rOXg5CG',
'spotify:artist:1n2pb9Tsfe4SwAjmUac6YT',
'spotify:artist:30e8DmahrEamvLbFRPdWmk',
'spotify:artist:7MynOMOTsGz8NFcPUImWAp',
'spotify:artist:72qxDp6R6v22pai96pZioz',
'spotify:artist:7dNsHhGeGU5MV01r06O8gK',
'spotify:artist:6h29EZDlc2NTvmCUe4yoXV',
'spotify:artist:5he5w2lnU9x7JFhnwcekXX',
'spotify:artist:4QQgXkCYTt3BlENzhyNETg',
'spotify:artist:7mldB9nEb6VC2cUj7EcgFw',
'spotify:artist:7jocoSCuCtpCxCI6IbP8ye',
'spotify:artist:2LIk90788K0zvyj2JJVwkJ',
'spotify:artist:5spEJXLwD1sKUdC2bnOHPg',
'spotify:artist:6p5JxpTc7USNnBnLzctyd4',
'spotify:artist:6lLYZf9GpSq6Pf09g2N2xK',
'spotify:artist:6UBA15slIuadJ8h2lPRPos',
'spotify:artist:4dBOV77d0Fy9KcTZkieXcu',
'spotify:artist:4XDxFeclKBCFPb30yQlRUH',
'spotify:artist:5ksRONqssB7BR161NTtJAm',
'spotify:artist:14ZxDAK6ITtZZqPdiWrvSn',
'spotify:artist:66lH4jAE7pqPlOlzUKbwA0',
'spotify:artist:2EExdpjU4SK3xnJHO5paJf',
'spotify:artist:4S9EykWXhStSc15wEx8QFK',
'spotify:artist:0R6Oa8qhUXdLRFL8Hiytgn',
'spotify:artist:0SD4eZCN4Kr0wQk56hCdh2',
'spotify:artist:6ZV6lGY0prbqpTD0Md8i29',
'spotify:artist:3RqBeV12Tt7A8xH3zBDDUF',
'spotify:artist:1uNFoZAHBGtllmzznpCI3s',
'spotify:artist:3ltFy7g6KKQPPttsdOMlq3',
'spotify:artist:0BvkDsjIUla7X0k6CSWh1I',
'spotify:artist:2ibPkysx2PXqWLmxFD7jSg',
'spotify:artist:0lJlKQvuM2Sd9DPPyUXcHg',
'spotify:artist:2JyWXPbkqI5ZJa3gwqVa0c',
'spotify:artist:4KWTAlx2RvbpseOGMEmROg',
'spotify:artist:2y8Jo9CKhJvtfeKOsYzRdT',
'spotify:artist:7mEIug7XUlQHikrFxjTWes',
'spotify:artist:6N1ycRllcq7fwHMdeTQD1z',
'spotify:artist:64mPnRMMeudAet0E62ypkx',
'spotify:artist:4aXXDj9aZnlshx7mzj3W1N',
'spotify:artist:5xenrbYhp4IRydrWpBe4Bx',
'spotify:artist:20p5D2KrE8CGuOjHtxsyTp',
'spotify:artist:1l40tPBtp1bmS1krmINRSD',
'spotify:artist:6RAiPa076RxBeTBxWO8Xfh',
'spotify:artist:2DaxqgrOhkeH0fpeiQq2f4',
'spotify:artist:5ZsFI1h6hIdQRw2ti0hz81',
'spotify:artist:1fa0cOhromAZdq2xRA4vv8',
'spotify:artist:2I1bnmb9VQEQGKHxvr0gSf',
'spotify:artist:1wvlC6NwleHt1iRD6d5X2C',
'spotify:artist:4d3yvTptO48nOYTPBcPFZC',
'spotify:artist:2wIVse2owClT7go1WT98tk',
'spotify:artist:3wyVrVrFCkukjdVIdirGVY',
'spotify:artist:4O15NlyKLIASxsJ0PrXPfz',
'spotify:artist:12mkoe71zJDWs4j0rCeXGR',
'spotify:artist:6pRi6EIPXz4QJEOEsBaA0m',
'spotify:artist:6wWVKhxIU2cEi0K81v7HvP',
'spotify:artist:4hqDqHtBlgxXpLXVYf7c8L',
'spotify:artist:2S9W9aSAd7e5mp8WqWxN2h',
'spotify:artist:7crPfGd2k81ekOoSqQKWWz',
'spotify:artist:4kEAjV4pCBOkoowYYQydvO',
'spotify:artist:6FbDoZnMBTdhhhLuJBOOqP',
'spotify:artist:2QmLFuIDtNDmmJY3OtvinN',
'spotify:artist:1TMMyiSsNzmRiZCAkVLVb2',
'spotify:artist:6UE7nl9mha6s8z0wFQFIZ2',
'spotify:artist:6r1Xmz7YUD4z0VRUoGm8XN',
'spotify:artist:4mOt6H7OnuqLnQrm1Bboc6',
'spotify:artist:50hFWG3MaUUX5phNrrVc97',
'spotify:artist:3g2kUQ6tHLLbmkV7T4GPtL',
'spotify:artist:3jdODvx7rIdq0UGU7BOVR3',
'spotify:artist:0NbfKEOTQCcwd6o7wSDOHI',
'spotify:artist:72NXpYBIaTfEeAAsxXLs0P',
'spotify:artist:2QMMktUMtwMSwUZATX1cRO',
'spotify:artist:5N7MwgMY3UYpFOnIHt1MXS',
'spotify:artist:0EdvGhlC1FkGItLOWQzG4J',
'spotify:artist:0VKh7CQDi9MkUvaBMoK1V0',
'spotify:artist:3tJnB0s6c3oXPq1SCCavnd',
'spotify:artist:1i4xi4ramy5BPYr2Bhf6sc',
'spotify:artist:6OWapcJm9xd55ci9CYbAuT',
'spotify:artist:34ABXKUj0gzq7R8vXjCaNj',
'spotify:artist:67FFKYikvTlvsPNk4NPOYJ',
'spotify:artist:5xQKoGD7Ql92fWd1uWwKkf',
'spotify:artist:0oSGxfWSnnOXhD2fKuz2Gy',
'spotify:artist:1Hsdzj7Dlq2I7tHP7501T4',
'spotify:artist:6vLlQYujOujIrm7zAKzEdG',
'spotify:artist:4Tw2N3wdvJPGEU7JqMxFfE',
'spotify:artist:3Fz2GbraVXhcpXnoi2Oe1r',
'spotify:artist:55vs7NT1KxcFjbMC4y202E',
'spotify:artist:0ZIwOAzDuGPspzK7yiTc4S',
'spotify:artist:4fxd5Ee7UefO4CUXgwJ7IP',
'spotify:artist:2AM4ilv6UzW0uMRuqKtDgN',
'spotify:artist:2jS7I1u7BpgWT9ssG62Zr1',
'spotify:artist:10Khz9BDdDT2mzm3330Cvu',
'spotify:artist:4jL1usPiGDK8TD46zJP4gu',
'spotify:artist:3crnzLy8R4lVwaigKEOz7V',
'spotify:artist:3JFsVIxOn7STeilPICkkB2',
'spotify:artist:7hJcb9fa4alzcOq3EaNPoG',
'spotify:artist:4sb7rZNN93BSS6Gqgepo4v',
'spotify:artist:5BcAKTbp20cv7tC5VqPFoC',
'spotify:artist:5sD1ZLf2dGQ9gQ3YJl1eAd',
'spotify:artist:64tNsm6TnZe2zpcMVMOoHL',
'spotify:artist:4iudEcmuPlYNdbP3e1bdn1',
'spotify:artist:2UazAtjfzqBF0Nho2awK4z',
'spotify:artist:5xUf6j4upBrXZPg6AI4MRK',
'spotify:artist:0iHb0mCbqZTYeb4y9Pirrd',
'spotify:artist:1w5Kfo2jwwIPruYS2UWh56',
'spotify:artist:4IVAbR2w4JJNJDDRFP3E83',
'spotify:artist:2HPaUgqeutzr3jx5a9WyDV',
'spotify:artist:1ZwdS5xdxEREPySFridCfh',
'spotify:artist:1HwM5zlC5qNWhJtM00yXzG',
'spotify:artist:1m24736Bdew1oQVxTePOCo',
'spotify:artist:50JJSqHUf2RQ9xsHs0KMHg',
'spotify:artist:2s6JabvZdqVQhfOsaxCSFX',
'spotify:artist:55CCS64lPz88twUDVquU1u',
'spotify:artist:694QW15WkebjcrWgQHzRYF',
'spotify:artist:1lZvg4fNAqHoj6I9N8naBM',
'spotify:artist:70kkdajctXSbqSMJbQO424',
'spotify:artist:00FQb4jTyendYWaN8pK0wa',
'spotify:artist:23zg3TcAtWQy7J6upgbUnj',
'spotify:artist:21E3waRsmPlU7jZsS13rcj',
'spotify:artist:7bXgB6jMjp9ATFy66eO08Z',
'spotify:artist:2iojnBLj0qIMiKPvVhLnsH',
'spotify:artist:3sMYEBy0CZFxedcnm9i9hf',
'spotify:artist:1pTuR132U5b4Rizal2Pr7m',
'spotify:artist:1YPIe9qIMa99O0a0OW47Be',
'spotify:artist:2M4Yt7oKGoYd0wqU44k4i2',
'spotify:artist:5CCf77sKGlTdfIiIDaVScj',
'spotify:artist:3tkZthBATD0S45hm1DmB2O',
'spotify:artist:4AwLiAGc1rRLrCepj5rdTI',
'spotify:artist:69Etpls5a2aATUkB6NInTY',
'spotify:artist:36eTeLNx8ErucrEyRm6JOo',
'spotify:artist:3U7k7hBsHF9dy4ELW53Wrv',
'spotify:artist:3Ay15wt0QChT4Kapsuw5Jt',
'spotify:artist:3THMgU4KdL7LlO5TEREs2g',
'spotify:artist:2BQVRw9md4UKcGUrDXABCD',
'spotify:artist:4K3DSWzghkGUcQOEZG9gpo',
'spotify:artist:4XwKU420OWVk38vz1KvH3M',
'spotify:artist:0WwSkZ7LtFUFjGjMZBMt6T',
'spotify:artist:0ym94xKp2PIOJtTZKpxbAa',
'spotify:artist:69GGBxA162lTqCwzJG5jLp',
'spotify:artist:23fqKkggKUBHNkbKtXEls4',
'spotify:artist:2sxmKe3CUrWnx7eoXMhOlW',
'spotify:artist:6PvvGcCY2XtUcSRld1Wilr',
'spotify:artist:6PvvGcCY2XtUcSRld1Wilr',
'spotify:artist:1UkHCANZmdkKyVNvTLMrcJ',
'spotify:artist:59wfkuBoNyhDMQGCljbUbA',
'spotify:artist:59wfkuBoNyhDMQGCljbUbA',
'spotify:artist:0u6b6Qa5VJEvr3SgzzeVOB',
'spotify:artist:5DYAABs8rkY9VhwtENoQCz',
'spotify:artist:4mITrNyXdPhy8phBHPgAHk',
'spotify:artist:6APm8EjxOHSYM5B4i3vT3q',
'spotify:artist:5wpEBloInversG3zp3CVAk',
'spotify:artist:6eJqAWJdd8JhAN1pQGie4r',
'spotify:artist:46gyXjRIvN1NL1eCB8GBxo',
'spotify:artist:4TqmaFJYM8SvuhrunxpmT3',
'spotify:artist:41qil2VaGbD194gaEcmmyx',
'spotify:artist:1mG23iQeR29Ojhq89D5gbh',
'spotify:artist:3TQ9JTBI2n2hfo7aRONEYV',
'spotify:artist:4046bnFxFJdLEtG7F2qXaV',
'spotify:artist:2zrZfs23sjuHDv4E6YRmNf',
'spotify:artist:7GXXMm3DB1VswVcuGyInUd',
'spotify:artist:7wqtxqI3eo7Gn1P7SpP6cQ',
'spotify:artist:5m7wCUhYhBh7A3A3YMxrbt',
'spotify:artist:2htToCUWzqrvOX6ISJY1nd',
'spotify:artist:2ye2Wgw4gimLv2eAKyk1NB',
'spotify:artist:5kFY4Bv5VRSBsM6CUYBoTC',
'spotify:artist:7xAcVHPiirnUqfdqo0USb1',
'spotify:artist:7gSjFKpVmDgC2MMsnN8CYq',
'spotify:artist:35GL8Cu2GKTcHzKGi75xl5',
'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH',
'spotify:artist:0PcIlEZa7rreM7729ot05g',
'spotify:artist:4TKTii6gnOnUXQHyuo9JaD',
'spotify:artist:1vnIL4DMlivP55ioM6KitW',
'spotify:artist:1YfEcTuGvBQ8xSD1f53UnK',
'spotify:artist:2Gop8SbSVt5SaJzuaRFa7D',
'spotify:artist:5fRiVl9fyhUEZhcpMyIxUG',
'spotify:artist:3gMaNLQm7D9MornNILzdSl',
'spotify:artist:2zSHqHILFTgz1xT3hopJQr',
'spotify:artist:6pgfx4K1p1kT0RewXjmLfk',
'spotify:artist:6ltcwvni6HdZAJaWhmIvNR',
'spotify:artist:45kyLFJs5SIEqyzfVopuSc',
'spotify:artist:6y8XlgIV8BLlIg1tT1R10i',
'spotify:artist:0Lf3tvw0Rzwr38KDrShstn',
'spotify:artist:7gjAu1qr5C2grXeQFFOGeh',
'spotify:artist:4lxfqrEsLX6N1N4OCSkILp',
'spotify:artist:7dWYWUbO68rXJOcyA7SpJk',
'spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS',
'spotify:artist:5INjqkS1o8h1imAzPqGZBb',
'spotify:artist:0kbYTNQb4Pb1rPbbaF0pT4',
'spotify:artist:3s2wTjWxK8NOX09dmsvVOh',
'spotify:artist:0CF71zaDOJWCynIkW9bSK8',
'spotify:artist:32WkQRZEVKSzVAAYqukAEA',
'spotify:artist:6scOultrkXrQsClcbGKM7e',
'spotify:artist:2mG8HHQ9S9kcbjcrb5N1FE',
'spotify:artist:53RkHTcl0SJZjpzqogkBf4',
'spotify:artist:1L0y9srZMyh9XUnYGv37IP',
'spotify:artist:0O0lrN34wrcuBenkqlEDZe',
'spotify:artist:0YrtvWJMgSdVrk3SfNjTbx',
'spotify:artist:5wbIWUzTPuTxTyG6ouQKqz',
'spotify:artist:2hnzQ6eCFkxUIPsVcsdj8A',
'spotify:artist:4MVyzYMgTwdP7Z49wAZHx0',
'spotify:artist:10rzK0sLCXsUlDhl8ed0MV',
'spotify:artist:21ysNsPzHdqYN2fQ75ZswG',
'spotify:artist:72JEwd3EjDEwTLypkFYJn9',
'spotify:artist:05oH07COxkXKIMt6mIPRee',
'spotify:artist:1vfezMIyCr4XUdYRaKIKi3',
'spotify:artist:601893mmW5hl1FBOykWZHG',
'spotify:artist:79FMDwzZQxHgSkIYBl3ODU',
'spotify:artist:5yhxqYI0JBwUKfXpSEjiM8',
'spotify:artist:7r0Otd9dEStSVIxJ02ml5a',
'spotify:artist:1EgGVV9cmmlLEsFlunjmvv',
'spotify:artist:3yY2gUcIsjMr8hjo51PoJ8',
'spotify:artist:78xUyw6FkVZrRAtziFdtdu',
'spotify:artist:0BkAYxgwF9VQiid4wI07yJ',
'spotify:artist:1JdeDdujDbS6ZeSlkXqx3a',
'spotify:artist:2yTUYhIf8fxptTIy3KLuJD',
'spotify:artist:3Isy6kedDrgPYoTS1dazA9',
'spotify:artist:6pXCjxMOBcWtvULYkFPVW6',
'spotify:artist:0KMLfCXYb3Dhf1hLpu8cVd',
'spotify:artist:3GMoVpWJy4smKuxFuFTwXC',
'spotify:artist:49qiE8dj4JuNdpYGRPdKbF',
'spotify:artist:5t28BP42x2axFnqOOMg3CM',
'spotify:artist:6gZq1Q6bdOxsUPUG1TaFbF',
'spotify:artist:7JDSHlDdVTo7aZKdQZ53Vf',
'spotify:artist:6rxiUsSvkn5yTs1kfuV8Le',
'spotify:artist:6nfYGe7IIuuP5bMY1jkJP6',
'spotify:artist:6om12Ev5ppgoMy3OYSoech',
'spotify:artist:6lx6XS8umFqYRVbBErx9fE',
'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB',
'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB',
'spotify:artist:7v4imS0moSyGdXyLgVTIV7',
'spotify:artist:1VbWUxZTRNY2gw3qZ1tg9W',
'spotify:artist:6ogn9necmbUdCppmNnGOdi',
'spotify:artist:2mpeljBig2IXLXRAFO9AAs',
'spotify:artist:0K7VN4aHxHcEb7PqkfoIVA',
'spotify:artist:6W1BHDF0T4a4KYcSwzD586',
'spotify:artist:0djV4iaxhNfYWpH60ia85o',
'spotify:artist:3X0tJzVYoWlfjLYI0Ridsw',
'spotify:artist:4NgNsOXSwIzXlUIJcpnNUp',
'spotify:artist:4MCBfE4596Uoi2O4DtmEMz',
'spotify:artist:45O9BwPMyywM755SYUK0sP',
'spotify:artist:6bOYtKnpLPQSfMpS2ilotK',
'spotify:artist:7ceUfdWq2t5nbatS6ollHh',
'spotify:artist:7K78lVZ8XzkjfRSI7570FF',
'spotify:artist:3naY4Tq98GRh9K8xOsINyv',
'spotify:artist:2dogRElUKV20C2khRHqjRc',
'spotify:artist:5VX8hxrcfJWwaTLiqGUHG3',
'spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq',
'spotify:artist:6KImCVD70vtIoJWnq6nGn3',
'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr',
'spotify:artist:5Rl15oVamLq7FbSb0NNBNy',
'spotify:artist:5y1gFSVnE4DGxUxHeKrGk6',
'spotify:artist:3vcFXwLEUdfWMu7gTQKyot',
'spotify:artist:5dZiRoY9Vpp6qrxqXNRwPE',
'spotify:artist:5338fdSzGNOFD4cxAxDTeq',
'spotify:artist:7x83XhcMbOTl1UdYsPTuZM',
'spotify:artist:3C4t1usFGoKHCSRdujm2GN',
'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H',
'spotify:artist:5lKZWd6HiSCLfnDGrq9RAm',
'spotify:artist:3r17AfJCCUqC9Lf0OAc73G',
'spotify:artist:3jtTd1E9mBxYYU987s32Fm',
'spotify:artist:4YfnJEbTz8QLD5FusYFwV6',
'spotify:artist:1hXjTjJzZTXDZ75AclOo6N',
'spotify:artist:35Uu85Pq33mK8x1jYqsHY2',
'spotify:artist:6uB4Ax6mlzohDTLqLOJ6Vl',
'spotify:artist:3tx8fyu3c4OBP5nejYtUOb',
'spotify:artist:0Z4CzYz9ieK8q9XiVMPkW5',
'spotify:artist:3gdbcIdNypBsYNu3iiCjtN',
'spotify:artist:4VmEWwd8y9MCLwexFMdpwt',
'spotify:artist:34EP7KEpOjXcM2TCat1ISk',
'spotify:artist:1tfXh4XxILEuMU2qZupFn8',
'spotify:artist:6V03b3Y36lolYP2orXn8mV',
'spotify:artist:1FizW0swe8ODtXvEOHXbHX',
'spotify:artist:0SdiiPkr02EUdekHZJkt58',
'spotify:artist:3BmGtnKgCSGYIUhmivXKWX',
'spotify:artist:0fgtHVpOPfXb07S8Jx443z',
'spotify:artist:3jc496ljiyrS3ECrD7QiqL',
'spotify:artist:4etuCZVdP8yiNPn4xf0ie5',
'spotify:artist:3nFB4eMP5gdqee2eQb8nZb',
'spotify:artist:13YoXxZt6SLg96dQE1vkHy',
'spotify:artist:3IhWQSrLj8EJjdvjFTpCyo',
'spotify:artist:0khzcQ6PQU5a6dl1Qss3pt',
'spotify:artist:4yiXkMbxDIZ3jgEG89NSC1',
'spotify:artist:6SFUC6ORDCIBqPssCBpeHT',
'spotify:artist:3XlIhgydjvC4EniPFZT20j',
'spotify:artist:7MSTO40bsb3DOJQT6xLsIc',
'spotify:artist:7dmeVSH4lJqxXU7C87dKIB',
'spotify:artist:4UXqAaa6dQYAk18Lv7PEgX',
'spotify:artist:6XyY86QOPPrYVGvF9ch6wz',
'spotify:artist:0blJzvevdXrp21YeI2vbco',
'spotify:artist:4yiQZ8tQPux8cPriYMWUFP',
'spotify:artist:5m8H6zSadhu1j9Yi04VLqD',
'spotify:artist:450o9jw6AtiQlQkHCdH6Ru',
'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ',
'spotify:artist:7gP3bB2nilZXLfPHJhMdvc',
'spotify:artist:4uBSazM6snEc9wCG3jMlYt',
'spotify:artist:5tTsrGPwQRWUsHR2Xf7Ke9',
'spotify:artist:42TFhl7WlMRXiNqzSrnzPL',
'spotify:artist:6ZLy8YFk7VbamsI5qbYXtw',
'spotify:artist:2enKa9Yqr6PZy3xGlxQEu7',
'spotify:artist:3KNZ9AOjBajjmr6ZR2VGfl',
'spotify:artist:70cRZdQywnSFp9pnc2WTCE',
'spotify:artist:08F3Y3SctIlsOEmKd6dnH8',
'spotify:artist:3gdIwZY6Q3RXhDteYr4ZvC',
'spotify:artist:0EmeFodog0BfCgMzAIvKQp',
'spotify:artist:5bTTx0CRvZj1kRJwUsWWYo',
'spotify:artist:5lJBrQQ88JjskJmJeVKX4F',
'spotify:artist:19I4tYiChJoxEO5EuviXpz',
'spotify:artist:4PPoI9LuYeFX8V674Z1R6l',
'spotify:artist:24DQLSng7bKZD4GXLIaQbv',
'spotify:artist:2GMH35k5oLCjzYpn5HbaD8',
'spotify:artist:6Tw1ktF4xMmzaLLbe98I2z',
'spotify:artist:21LGsW7bziR4Ledx7WZ1Wf',
'spotify:artist:0TGsWvnbtExsbGUSWtXENE',
'spotify:artist:67ea9eGLXYMsO2eYQRui3w',
'spotify:artist:4xXCRXOfQKQ2gjWxNhNzYW',
'spotify:artist:2VrP5BkyzDhY43dWMlaYjD',
'spotify:artist:0grdhNhiRLFBaFVyybqsj6',
'spotify:artist:5gAbCaW9OmfIqqZGLpKb3f',
'spotify:artist:62olK5zZHSgFUXGDykgBL8',
'spotify:artist:5W0GCoUUwU3SF6qpJanlug',
'spotify:artist:3eFo5fMv53RYZBYlvT5Z6a',
'spotify:artist:44y3DMkP83lyU25GaP8y7U',
'spotify:artist:5wPoxI5si3eJsYYwyXV4Wi',
'spotify:artist:3CkvROUTQ6nRi9yQOcsB50',
'spotify:artist:3qbnxnvUqR14MJ9g8QwZJK',
'spotify:artist:6NUhQz7eAEsZvjEHTKHux9',
'spotify:artist:2FXC3k01G6Gw61bmprjgqS',
'spotify:artist:2hlmm7s2ICUX0LVIhVFlZQ',
'spotify:artist:699OTQXzgjhIYAHMy9RyPD',
'spotify:artist:4Q5sPmM8j4SpMqL4UA1DtS',
'spotify:artist:3MZsBdqDrRTJihTHQrO6Dq',
'spotify:artist:4cORFIWQylRGyJX9lYnqk9',
'spotify:artist:7o95ZoZt5ZYn31e9z1Hc0a',
'spotify:artist:3nnQpaTvKb5jCQabZefACI',
'spotify:artist:7w29UYBi0qsHi5RTcv3lmA',
'spotify:artist:19Fi1Rj7kk8kyiwxpXy3yM',
'spotify:artist:6vbY3hOaCAhC7VjucswgdS',
'spotify:artist:2DWQX92uew7DlsgN0yD775',
'spotify:artist:4rulVipLmUL2Mg67SIUmCO',
'spotify:artist:7MbmKsnvXjl4GA7Dr27kko',
'spotify:artist:72Nhcx7prNk2ZCxhx0Y5es',
'spotify:artist:1SaELUYn7deVoQ9kGDGUD9',
'spotify:artist:6H1RjVyNruCmrBEWRbD0VZ',
'spotify:artist:1Fmb52lZ6Jv7FMWXXTPO3K',
'spotify:artist:63yrD80RY3RNEM2YDpUpO8',
'spotify:artist:63yrD80RY3RNEM2YDpUpO8',
'spotify:artist:02NfyD6AlLA12crYzw5YcR',
'spotify:artist:2VYQTNDsvvKN9wmU5W7xpj',
'spotify:artist:6GbCJZrI318Ybm8mY36Of5',
'spotify:artist:5lwmRuXgjX8xIwlnauTZIP',
'spotify:artist:5BcZ22XONcRoLhTbZRuME1',
'spotify:artist:26s8LSolLfCIY88ysQbIuT',
'spotify:artist:4Y7tXHSEejGu1vQ9bwDdXW',
'spotify:artist:0fyqyjD7pbaVzbu94ylWQR',
'spotify:artist:7emRV8AluG3d4e5T0DZiK9',
'spotify:artist:2GHclqNVjqGuiE5mA7BEoc',
'spotify:artist:1pPmIToKXyGdsCF6LmqLmI',
'spotify:artist:7wlFDEWiM5OoIAt8RSli8b',
'spotify:artist:5qwERxFiC8zCJvnVI8Zkl9',
'spotify:artist:3RFGnJaDVOyxL9YcFaKatu',
'spotify:artist:7gZfnEnfiaHzxARJ2LeXrf',
'spotify:artist:1URnnhqYAYcrqrcwql10ft',
'spotify:artist:31W5EY0aAly4Qieq6OFu6I',
'spotify:artist:0iEtIxbK0KxaSlF7G42ZOp',
'spotify:artist:2qFr8w5sWUITRlzZ9kZotF',
'spotify:artist:7b85ve82Sh36a3UAx74wut',
'spotify:artist:6ZPbX2Lxd40e2UVkXpyxvI',
'spotify:artist:3BVkDHWRvLJEyKdvhLbjsq',
'spotify:artist:3UbyYnvNIT5DFXU4WgiGpP',
'spotify:artist:56oDRnqbIiwx4mymNEv7dS',
'spotify:artist:51Blml2LZPmy7TTiAg47vQ',
'spotify:artist:4dwdTW1Lfiq0cM8nBAqIIz',
'spotify:artist:246dkjvS1zLTtiykXe5h60',
'spotify:artist:27T030eWyCQRmDyuvr1kxY',
'spotify:artist:568ZhdwyaiCyOGJRtNYhWf',
'spotify:artist:7xTKLpo7UCzXSnlH7fOIoM',
'spotify:artist:16nn7kCHPWIB6uK09GQCNI',
'spotify:artist:6rJqqRce0Kvo2dJUXoHleC',
'spotify:artist:336vr2M3Va0FjyvB55lJEd',
'spotify:artist:0u2FHSq3ln94y5Q57xazwf',
'spotify:artist:4Ud7lY9V8pOyydumajSW3O',
'spotify:artist:1VQ7baxc9Okx2YuRnpKMMR',
'spotify:artist:1HJlSybDbI7ASahsDPLXac',
'spotify:artist:3tC7pAMcEoU8En89pu1i1e',
'spotify:artist:4IliztYDlfMvzQzbx50o60',
'spotify:artist:46lnlnlU0dXTDpoAUmH6Qx',
'spotify:artist:5uYXMC13cIUulobh204QuK',
'spotify:artist:5tth2a3v0sWwV1C7bApBdX',
'spotify:artist:34jw2BbxjoYalTp8cJFCPv',
'spotify:artist:162DCkd8aDKwvjBb74Gu8b',
'spotify:artist:3Ao7NH7lRyQAeKQg2mlTcO',
'spotify:artist:1DXylZlWbVvlckNqwvjTEt',
'spotify:artist:6sq1yF0OZEWA4xoXVKW1L9',
'spotify:artist:5J6L7N6B4nI1M5cwa29mQG',
'spotify:artist:0uNC9XuH437fKCCMuzvSks',
'spotify:artist:4pr7J7wzgObkE3DD3Izi7q',
'spotify:artist:2cGwlqi3k18jFpUyTrsR84',
'spotify:artist:2FjkZT851ez950cyPjeYid',
'spotify:artist:1XpDYCrUJnvCo9Ez6yeMWh',
'spotify:artist:3Z02hBLubJxuFJfhacLSDc',
'spotify:artist:5xWPOujQqd4wXyB08slZ9Z',
'spotify:artist:6YHEMoNPbcheiWS2haGzkn',
'spotify:artist:0B6QEFtRnneEzb4iqjI0Nw',
'spotify:artist:6h2bWHWTJL38N8dqocVaif',
'spotify:artist:67nwj3Y5sZQLl72VNUHEYE',
'spotify:artist:20sxb77xiYeusSH8cVdatc',
'spotify:artist:0QHgL1lAIqAw0HtD7YldmP',
'spotify:artist:7A9yZMTrFZcgEWAX2kBfK6',
'spotify:artist:7uhvDINTTiD0XBrP9fquN1',
'spotify:artist:7y70dch6JuuuNnwlsOQvwW',
'spotify:artist:0bc7dhY6YXL35m1G0jVwuW',
'spotify:artist:3OyGv7XUYQwQgECYSzJhyO',
'spotify:artist:3g229vTyhsdwm9SO2p1FPL',
'spotify:artist:7FIoB5PHdrMZVC3q2HE5MS',
'spotify:artist:2hO4YtXUFJiUYS2uYFvHNK',
'spotify:artist:1sXbwvCQLGZnaH0Jp2HTVc',
'spotify:artist:0X2BH1fck6amBIoJhDVmmJ',
'spotify:artist:6M2wZ9GZgrQXHCFfjv46we',
'spotify:artist:1zNqDE7qDGCsyzJwohVaoX',
'spotify:artist:2gsggkzM5R49q6jpPvazou',
'spotify:artist:4npEfmQ6YuiwW1GpUmaq3F',
'spotify:artist:6MDME20pz9RveH9rEXvrOM',
'spotify:artist:1Xylc3o4UrD53lo9CvFvVg',
'spotify:artist:5obkpruq3y0AJFeZnNLheK',
'spotify:artist:55bGuHb50r5c0PeqqMeNBV',
'spotify:artist:2D38buglrcCl9TMZ45tkh1',
'spotify:artist:2JSjCHK79gdaiPWdKiNUNp',
'spotify:artist:5y2Xq6xcjJb2jVM54GHK3t',
'spotify:artist:0AD4odMWVQ2wUSlgxOB5Rl',
'spotify:artist:1Zq8pfBl4ejCMrWdeAdphc',
'spotify:artist:3jksrX4oBklxR78ft8gv3j',
'spotify:artist:6Ha4aES39QiVjR0L2lwuwq',
'spotify:artist:0eDvMgVFoNV3TpwtrVCoTj',
'spotify:artist:2DnqqkzzDKm3vAoyHtn8So',
'spotify:artist:1UTPBmNbXNTittyMJrNkvw',
'spotify:artist:0zlJDQgqnr9AocrH9gs1Es',
'spotify:artist:24Wa5wIZIo1sPkzVGP0B5p',
'spotify:artist:2FcC4sDMXme2ziI7tGKMK8',
'spotify:artist:4l2d1CRNjXgG4RBoA89Kle',
'spotify:artist:1sCa5KANHoHo4NuGextoI9',
'spotify:artist:2iVKGetYUZQElKZKWECyF4',
'spotify:artist:7I45cJlgpJa5SLVWqocmjb',
'spotify:artist:0qAmeOULjwn04k6jMMeDyr',
'spotify:artist:5s6TJEuHTr9GR894wc6VfP',
'spotify:artist:64CuUOOirKmdAYLQSfaOyr',
'spotify:artist:6D7h7R79IZjqJC2GM2wzyY',
'spotify:artist:7290H8m1Dwt8G7jm1y9CQx',
'spotify:artist:6ROyXB8NDG0fA78S46JdTB',
'spotify:artist:0MvSBMGRQJY3mRwIbJsqF1',
'spotify:artist:1YLsqPcFg1rj7VvhfwnDWm',
'spotify:artist:03r4iKL2g2442PT9n2UKsx',
'spotify:artist:5Pb27ujIyYb33zBqVysBkj',
'spotify:artist:6ns6XAOsw4B0nDUIovAOUO',
'spotify:artist:72eP0W3rIhkxd0NHGg4w4u',
'spotify:artist:4LEiUm1SRbFMgfqnQTwUbQ',
'spotify:artist:6hCsqVHnBo1BVQWuIjRMkL',
'spotify:artist:3Y7RZ31TRPVadSFVy1o8os',
'spotify:artist:1LB8qB5BPb3MHQrfkvifXU',
'spotify:artist:1Ha0Fz4i0d4gu5fZbhBCtH',
'spotify:artist:2cNV8vNe2fPFf6OfGOBbSS',
'spotify:artist:6aZyMrc4doVtZyKNilOmwu',
'spotify:artist:0hYxQe3AK5jBPCr5MumLHD',
'spotify:artist:7nkLRaWHImCvWGHdNGnhVE',
'spotify:artist:0MAvx5yzdhylg2ztJC3MD3',
'spotify:artist:3GzWhE2xadJiW8MqRKIVSK',
'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg',
'spotify:artist:0A0FS04o6zMoto8OKPsDwY',
'spotify:artist:2tIP7SsRs7vjIcLrU85W8J',
'spotify:artist:47zz7sob9NUcODy0BTDvKx',
'spotify:artist:35U9lQaRWSQISxQAB94Meo',
'spotify:artist:70ZTdbPEcEugBNay4MvxfL',
'spotify:artist:0cA5Tg15TwARIRZeiNT1RO',
'spotify:artist:07YZf4WDAMNwqr4jfgOZ8y',
'spotify:artist:0du5cEVh5yTK9QJze8zA0C',
'spotify:artist:6uothxMWeLWIhsGeF7cyo4',
'spotify:artist:5KDIH2gF0VpelTqyQS7udb',
'spotify:artist:3dXaa6jwM7B52GZpaJEIr5',
'spotify:artist:2xiIXseIJcq3nG7C8fHeBj',
'spotify:artist:5BtHciL0e0zOP7prIHn3pP',
'spotify:artist:6deZN1bslXzeGvOLaLMOIF',
'spotify:artist:2RTUTCvo6onsAnheUk3aL9',
'spotify:artist:70BYFdaZbEKbeauJ670ysI',
'spotify:artist:3YVZDRxTrSOCR1TPDw0nBm',
'spotify:artist:6S2OmqARrzebs0tKUEyXyp',
'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF',
'spotify:artist:6dJeKm76NjfXBNTpHmOhfO',
'spotify:artist:0p4nmQO2msCgU4IF37Wi3j',
'spotify:artist:7FtVJzRtpQpU61nBwB7cKN',
'spotify:artist:3ngKsDXZAssmljeXCvEgOe',
'spotify:artist:126FigDBtqwS2YsOYMTPQe',
'spotify:artist:55VydwMyCuGcavwPuhutPL',
'spotify:artist:1SCWcYnoEh1978ycTDhZoW',
'spotify:artist:7oWSqrgMuIEyH9qp5nu2e5',
'spotify:artist:6WVbvBEbpy43tg5a1RwvFE',
'spotify:artist:1lqW59DUEKqvcHc8mVWBtH',
'spotify:artist:4UPcJIhr5K5fPsm4itqT7E',
'spotify:artist:1B8ySGDAiXTCvnJNH4HSCG',
'spotify:artist:2P3cjUru4H3fhSXXNxE9kA',
'spotify:artist:4UGMQyNcbGHYg5CDMKkSw3',
'spotify:artist:1QtIfAa6y7w2JhxYJhYeUG',
'spotify:artist:296LotOmbj7eOX7r9l46fK',
'spotify:artist:6O74knDqdv3XaWtkII7Xjp',
'spotify:artist:0EVUivUkugMtNF09L4QBMH',
'spotify:artist:5VfqJBmXcf6ZqXoGij5qTE',
'spotify:artist:1mFX1QlezK1lNPKQJkhwWb',
'spotify:artist:1eNkUXHPaXyuyC8NAgzykK',
'spotify:artist:7uwPrVdjss0FyCCTzpLx5A',
'spotify:artist:7u1mqP3ykglpCB2c1p1p5I',
'spotify:artist:4wM29TDTr3HI0qFY3KoSFG',
'spotify:artist:2C8mKwk3z5DtqLcSwUdYJ9',
'spotify:artist:0A7g2YbCA9FlyZvAG6VmKP',
'spotify:artist:4BoRxUdrcgbbq1rxJvvhg9',
'spotify:artist:4BoRxUdrcgbbq1rxJvvhg9',
'spotify:artist:4BoRxUdrcgbbq1rxJvvhg9',
'spotify:artist:4463nfFMmK1cwAWBQDwT5e',
'spotify:artist:13y0kncDD4J9wxCyfKr10W',
'spotify:artist:6jGMq4yGs7aQzuGsMgVgZR',
'spotify:artist:7s6c1gQXQCakbFxmgKAQDV',
'spotify:artist:74IEeKcuS34kF2TjOigXra',
'spotify:artist:0sD8Amms4kSxs5tBV4CUmR',
'spotify:artist:2ndyVAdV9UqF1XjyTJt484',
'spotify:artist:5r0KYGxdIZEHZ6z6XbkVbo',
'spotify:artist:0FcmRWJUfnCuwoXE1H2b0H',
'spotify:artist:45S8rtgQSxj1a0tzCNhSeD',
'spotify:artist:3r9TXuXfOUxXjgYgAR0fP8',
'spotify:artist:1wiBLzTI7z9RUwEpNPdFT6',
'spotify:artist:4dpARuHxo51G3z768sgnrY',
'spotify:artist:3LIJJJkO7R5RasRwt7xIn5',
'spotify:artist:4135eJjHCRw5SuVzLfzR0x',
'spotify:artist:4MptlUxSUHf2QAW9JMrv4W',
'spotify:artist:19ra5tSw0tWufvUp8GotLo',
'spotify:artist:5GtMEZEeFFsuHY8ad4kOxv',
'spotify:artist:378dH6EszOLFShpRzAQkVM',
'spotify:artist:2iHoAyWMSBlE0ynGkTloCn',
'spotify:artist:58Dx4HPzeOO3dbpD9YYEes',
'spotify:artist:5Y8EphH8Vdqu5SLj6K5vjj',
'spotify:artist:4TROLx8HvrnfGGNo2ngL2N',
'spotify:artist:15UsOTVnJzReFVN1VCnxy4',
'spotify:artist:3qm84nBOXUEQ2vnTfUTTFC',
'spotify:artist:5fSVkkuijRIpxuvSqysYiP',
'spotify:artist:1moxjboGR7GNWYIMWsRjgG',
'spotify:artist:163tK9Wjr9P9DmM0AVK7lm',
'spotify:artist:4NHQUGzhtTLFvgF5SZesLK',
'spotify:artist:07QEuhtrNmmZ0zEcqE9SF6',
'spotify:artist:6fvN9GmMCVKb5LY0WsnjFP',
'spotify:artist:3XsgWn63EnA4wYZBjVyxjf',
'spotify:artist:7CHilrn81OdYjkh4uSVnYM',
'spotify:artist:0KyCXNSa7ZMb5LydfKbLG3',
'spotify:artist:3Xt3RrJMFv5SZkCfUE8C1J',
'spotify:artist:0k7Xl1pqI3tu8sSEjo5oEg',
'spotify:artist:6vdMPayKk8YJxxeNP5oMCb',
'spotify:artist:5eAWCfyUhZtHHtBdNk56l1',
'spotify:artist:0TF2NxkJZPQoX1H53rEFM1',
'spotify:artist:3RwQ26hR2tJtA8F9p2n7jG',
'spotify:artist:0h9smro0z3HqUbD94jotU8',
'spotify:artist:20wkVLutqVOYrc0kxFs7rA',
'spotify:artist:1E4r4z7ivGUcSAnCbINJ9v',
'spotify:artist:4ERtgeBbWRkFzIz6LaFCeY',
'spotify:artist:0f8MDDzIc6M4uH1xH0o0gy',
'spotify:artist:2hGh5VOeeqimQFxqXvfCUf',
'spotify:artist:2EfG2EoT8GFJrMiilbTVl2',
'spotify:artist:08DLZqQd6XDAVteF9nSEJ1',
'spotify:artist:4RVnAU35WRWra6OZ3CbbMA',
'spotify:artist:5z1VAFwT35EVvCp1XlZZuL',
'spotify:artist:0rpMdBzQXf7aYRnu5fDBJy',
'spotify:artist:2zFZiWQJFFshzojycnXoTL',
'spotify:artist:6cjSmkVvMvyE6tCAo1M9Is',
'spotify:artist:0iVrCROxeyon7MZUW3MfzT',
'spotify:artist:36HvjE9pV91jEjO1qBRidS',
'spotify:artist:6olE6TJLqED3rqDCT0FyPh',
'spotify:artist:0UKfenbZb15sqhfPC6zbt3',
'spotify:artist:5gznATMVO85ZcLTkE9ULU7',
'spotify:artist:0Je74SitssvJg1w4Ra2EK7',
'spotify:artist:3pHeBYl1yujXcZqqfF1UyQ',
'spotify:artist:1YEGETLT2p8k97LIo3deHL',
'spotify:artist:02da1vDJ2hWqfK7aJL6SJm',
'spotify:artist:0gDg7FEsF4Y1jWddJJgcn4',
'spotify:artist:7ohlPA8dRBtCf92zaZCaaB',
'spotify:artist:2zsLaQhHFdQPdFXqG1nRmM',
'spotify:artist:6KOqPxwfNAmZPkiCnDE9yT',
'spotify:artist:4uYwLU7k03RCQSRXGtQGg0',
'spotify:artist:01WjpKiWVNurV5hjIadB8C',
'spotify:artist:2tFN9ubMXEhdAQvdQxcsma',
'spotify:artist:7hEbJvh3BAedDS2vVJvx0B',
'spotify:artist:5ugjlc51azqrMoBLLbOnIg',
'spotify:artist:3RNrq3jvMZxD9ZyoOZbQOD',
'spotify:artist:3HVdAiMNjYrQIKlOGxoGh5',
'spotify:artist:2Pfv2w8a20xzC7Dr7QXRqM',
'spotify:artist:165ZgPlLkK7bf5bDoFc6Sb',
'spotify:artist:5imUS9dQyCbAjUEJJ9QyWC',
'spotify:artist:05fG473iIaoy82BF1aGhL8',
'spotify:artist:5os0Ltvz8Q8BvXOPOd1frx',
'spotify:artist:3Z2jmNAP2UaGa8lPpi54wD',
'spotify:artist:2QsynagSdAqZj3U9HgDzjD',
'spotify:artist:08GQAI4eElDnROBrJRGE0X',
'spotify:artist:0S1SjYebdSaiMpI8xSW2Wb',
'spotify:artist:2YdVmtVBpIrv0N6WiBzSqm',
'spotify:artist:61UT1Zj9dFgPAjZfwnsqsb',
'spotify:artist:5qUHCOUsvVKUpVrMMmztMK',
'spotify:artist:5jNGQ7VOU87x5402JjhTtd',
'spotify:artist:3J9tQvcK0bY3CcVcgRELxH',
'spotify:artist:2yaixhgm3yXxjhJAH8SZy3',
'spotify:artist:6mQfAAqZGBzIfrmlZCeaYT',
'spotify:artist:46CH1Gp8l8QVly8bpG9JFG',
'spotify:artist:2XzHxDFTNywDd5ReMhgxkg',
'spotify:artist:0ty0xha1dbprYIUAQufkFn',
'spotify:artist:6EFL6smxzRtpwlQvZFldUc',
'spotify:artist:6RJYPm4gcrIuOjQiqu9VXu',
'spotify:artist:0ECwFtbIWEVNwjlrfc6xoL',
'spotify:artist:3IYUhFvPQItj6xySrBmZkd',
'spotify:artist:3IYUhFvPQItj6xySrBmZkd',
'spotify:artist:4CxobvwTpmfpIEbkYh4pAb',
'spotify:artist:2lolQgalUvZDfp5vvVtTYV',
'spotify:artist:2yzxX2DI9LFK8VFTyW2zZ8',
'spotify:artist:5alRKXufstWwyHsyQbpi9n',
'spotify:artist:1Bd4UVlqlaKEXYRG3wgrCK',
'spotify:artist:3RGLhK1IP9jnYFH4BRFJBS',
'spotify:artist:1SImpQO0GbjRgvlwCcCtFo',
'spotify:artist:1AhYnfs5mzb3ennyLFXlYD',
'spotify:artist:29kkCKKGXheHuoO829FxWK',
'spotify:artist:6HySSUk2KthFj3bd9R61ql',
'spotify:artist:2iEvnFsWxR0Syqu2JNopAd',
'spotify:artist:3zxKH0qp3nBCuPZCZT5Vaf',
'spotify:artist:3CygdxquGHurS7f9LjNLkv',
'spotify:artist:6eEsIl4wosvhuWI0mfXxAF',
'spotify:artist:7oPftvlwr6VrsViSDV7fJY',
'spotify:artist:3j7Sw80qzL2FJlryUhI3wG',
'spotify:artist:4V8U8U6LwsHGyRTLCt9t19',
'spotify:artist:4sD9znwiVFx9cgRPZ42aQ1',
'spotify:artist:7oPgCQqMMXEXrNau5vxYZP',
'spotify:artist:1KsASRNugxU85T0u6zSg32',
'spotify:artist:7ndzHjxbErIwvwnEUewMWe',
'spotify:artist:5nPz5R1FDPTMI3Dw4JnTGb',
'spotify:artist:4mar1GMMEhvGyJdWagu6KS',
'spotify:artist:1FE0rls8gfQT3laAeRYNgl',
'spotify:artist:32vWCbZh0xZ4o9gkz4PsEU',
'spotify:artist:4akybxRTGHJZ1DXjLhJ1qu',
'spotify:artist:6qI4LTzMRpTxRzMZPvv2C6',
'spotify:artist:5IFCkqu9J6xdWeYMk5I889',
'spotify:artist:4PpmBoqphQusNFsxuVKb6j',
'spotify:artist:7krx6UBDKLwE0q3s3fesqF',
'spotify:artist:19y5MFBH7gohEdGwKM7QsP',
'spotify:artist:46SHBwWsqBkxI7EeeBEQG7',
'spotify:artist:33ScadVnbm2X8kkUqOkC6Z',
'spotify:artist:6Iz3eq2aQGFf7TbGT2iahL',
'spotify:artist:1Yox196W7bzVNZI7RBaPnf',
'spotify:artist:3JysSUOyfVs1UQ0UaESheP',
'spotify:artist:1IQ2e1buppatiN1bxUVkrk',
'spotify:artist:7u6FSuqT3SWE2kO8rNVjfp',
'spotify:artist:5ZS223C6JyBfXasXxrRqOk',
'spotify:artist:7qG3b048QCHVRO5Pv1T5lw',
'spotify:artist:6S58b0fr8TkWrEHOH4tRVu',
'spotify:artist:1TJbmc7jTpw78GKCiMpvDh',
'spotify:artist:4njdEjTnLfcGImKZu1iSrz',
'spotify:artist:5aYyPjAsLj7UzANzdupwnS',
'spotify:artist:1xgFexIwrf2QjbU0buCNnp',
'spotify:artist:1UUYAQ9LiRsZF0ZukQNWXM',
'spotify:artist:4tpUmLEVLCGFr93o8hFFIB',
'spotify:artist:74ASZWbe4lXaubB36ztrGX',
'spotify:artist:74ASZWbe4lXaubB36ztrGX',
'spotify:artist:0SCbttzoZTnLFebDYmAWCm',
'spotify:artist:32HPpJAhgnGPT3V7UOggzi',
'spotify:artist:6zvul52xwTWzilBZl6BUbT',
'spotify:artist:3lzlUccNrekC1oFPCjNIOX',
'spotify:artist:3qtcbDvdIgQOXkPbtRvBaQ',
'spotify:artist:3rxIQc9kWT6Ueg4BhnOwRK',
'spotify:artist:0WjkBDqno4HbjwNDqyMgVa',
'spotify:artist:3TiISqKS6ESlMQ4WFfZJw2',
'spotify:artist:2kuNswDC82PL9xRbfaZJaS',
'spotify:artist:4asCC4oxQcDzFXhCth2SgQ',
'spotify:artist:5PokPZn11xzZXyXSfnvIM3',
'spotify:artist:7lzordPuZEXxwt9aoVZYmG',
'spotify:artist:5NGO30tJxFlKixkPSgXcFE',
'spotify:artist:6vWDO969PvNqNYHIOW5v0m',
'spotify:artist:4b0WsB47XCa9F83BmwQ7WX',
'spotify:artist:2O0Hw1WSMbskB5tD9aWah3',
'spotify:artist:47opbYEKDjXnRk9uLscp11',
'spotify:artist:1cHFz6lrt7KAsBV8j2Ny1g',
'spotify:artist:0dlOr0VIysztGWvU1dpjmP',
'spotify:artist:3TW7S3OjH8GmXtfvMHzOWb',
'spotify:artist:15jBNfRUPUgdZOTSaNoF5l',
'spotify:artist:3kUKwTJdH8FuWzF8p6Dg9E',
'spotify:artist:48YxSlb23RAaCd4RyHcV9V',
'spotify:artist:3utxjLheHaVEd9bPjQRsy8',
'spotify:artist:6XJ5IHDtW2OkjtwVllkANt',
'spotify:artist:7iMvwE8qANp3aIfAGKEAwS',
'spotify:artist:2yrbLiuBmc9j81lTX3XUuI',
'spotify:artist:0JDkhL4rjiPNEp92jAgJnS',
'spotify:artist:4vpDg7Y7fU982Ds30zawDA',
'spotify:artist:5nCi3BB41mBaMH9gfr6Su0',
'spotify:artist:5Pwc4xIPtQLFEnJriah9YJ',
'spotify:artist:3FUY2gzHeIiaesXtOAdB7A',
'spotify:artist:3VNITwohbvU5Wuy5PC6dsI',
'spotify:artist:0FrpdcVlJQqibaz5HfBUrL',
'spotify:artist:5ujCegv1BRbEPTCwQqFk6t',
'spotify:artist:51l0uqRxGaczYr4271pVIC',
'spotify:artist:6Hizgjo92FnMp8wGaRUNTn',
'spotify:artist:4FAEZeJcsYYBkNq2D3KGTV',
'spotify:artist:16oZKvXb6WkQlVAjwo2Wbg',
'spotify:artist:6kACVPfCOnqzgfEF5ryl0x',
'spotify:artist:5ypxebeHEIXjMtJb17uJlI',
'spotify:artist:0lZoBs4Pzo7R89JM9lxwoT',
'spotify:artist:7guDJrEfX3qb6FEbdPA5qi',
'spotify:artist:5bDxAyJiTYBat1YnFJhvEK',
'spotify:artist:7J7YRBSoiXpdTmcbkKZL2C',
'spotify:artist:3fMbdgg4jU18AjLCKBhRSm',
'spotify:artist:2EURsXo9qlt1aMWlviGCRi',
'spotify:artist:6HX8AbXUFaYRtlqKb4CCo0',
'spotify:artist:67BqBdApdbZ0onglFqRoTg',
'spotify:artist:12Chz98pHFMPJEknJQMWvI',
'spotify:artist:2qk9voo8llSGYcZ6xrBzKx',
'spotify:artist:3TNt4aUIxgfy9aoaft5Jj2',
'spotify:artist:6yrBBtqX2gKCHCrZOYBDrB',
'spotify:artist:7Ln80lUS6He07XvHI8qqHH',
'spotify:artist:2M4KXzgedR4EKG4qgTeDKa',
'spotify:artist:1bs7HoMkSyQwcobCpE9KpN',
'spotify:artist:1bs7HoMkSyQwcobCpE9KpN',
'spotify:artist:320EPCSEezHt1rtbfwH6Ck',
'spotify:artist:10exVja0key0uqUkk6LJRT',
'spotify:artist:3gd8FJtBJtkRxdfbTu19U2',
'spotify:artist:6GpuL6WNTOVhmMl6lZbiKS',
'spotify:artist:4wQ3PyMz3WwJGI5uEqHUVR',
'spotify:artist:49DW3KvkyjHO35mK1JnSyS',
'spotify:artist:3oDbviiivRWhXwIE8hxkVV',
'spotify:artist:3JsMj0DEzyWc0VDlHuy9Bx',
'spotify:artist:57ALvbCBaCkNlgTOSiUPdT',
'spotify:artist:4nlqDmbzFzbNITaqjJv7D7',
'spotify:artist:53QzNeFpzAaXYnrDBbDrIp',
'spotify:artist:68kACMx6A3D2BYiO056MeQ',
'spotify:artist:438JBZR1AR0l04AzcYW9gy',
'spotify:artist:59QxeZBL6k9L4oJBGSyukd',
'spotify:artist:0WgyCbru4tXnMsbTmX4mFw',
'spotify:artist:09hVIj6vWgoCDtT03h8ZCa',
'spotify:artist:1Z8ODXyhEBi3WynYw0Rya6',
'spotify:artist:5IDs1CK15HegSAhGEbSYXo',
'spotify:artist:2idymNTKUdnTxforkb12Mw',
'spotify:artist:6A43Djmhbe9100UwnI7epV',
'spotify:artist:3jOstUTkEu2JkjvRdBA5Gu',
'spotify:artist:2jw70GZXlAI8QzWeY2bgRc',
'spotify:artist:4F84IBURUo98rz4r61KF70',
'spotify:artist:1Y0IXModT7x4Vtj1gu3TeE',
'spotify:artist:6eMlKSBFAoXVJLoeHmwKEj',
'spotify:artist:6DCIj8jNaNpBz8e5oKFPtp',
'spotify:artist:047D9GgqEzyMoULd2oKr7G',
'spotify:artist:2sIx6SmAMw9IBySG3Uj0jf',
'spotify:artist:4OGuNAnRFWZOgOA2d51taz',
'spotify:artist:38h03gA85YYPeDPd9ER9rT',
'spotify:artist:6twIAGnYuIT1pncMAsXnEm',
'spotify:artist:7nwUJBm0HE4ZxD3f5cy5ok',
'spotify:artist:3Ayl7mCk0nScecqOzvNp6s',
'spotify:artist:26T3LtbuGT1Fu9m0eRq5X3',
'spotify:artist:5fbhwqYYh4YwUoEs582mq5',
'spotify:artist:0W498bDDNlJIrYMKXdpLHA',
'spotify:artist:1DpIDwg8FGD50N9Tfunfsf',
'spotify:artist:0rXI0q8Cahq6numvPlloaq',
'spotify:artist:7uykqYYhDUugFJgKZWXIWq',
'spotify:artist:3koiLjNrgRTNbOwViDipeA',
'spotify:artist:5a2EaR3hamoenG9rDuVn8j',
'spotify:artist:5a2EaR3hamoenG9rDuVn8j',
'spotify:artist:6V6WCgi7waF55bJmylC4H5',
'spotify:artist:6gkWznnJkdkwRPVcmnrays',
'spotify:artist:4apX9tIeHb85yPyy4F6FJG',
'spotify:artist:0Nn9YwJzcaeuU1jJL06e3r',
'spotify:artist:1Vvvx45Apu6dQqwuZQxtgW',
'spotify:artist:4rB5wLJLaXdMrXaYsOYLmK',
'spotify:artist:31zQEL3ARiYe63jn3jtM18',
'spotify:artist:3XHO7cRUPCLOr6jwp8vsx5',
'spotify:artist:74oJ4qxwOZvX6oSsu1DGnw',
'spotify:artist:3eskO5m0H4yiF64vRySBjr',
'spotify:artist:1SQRv42e4PjEYfPhS0Tk9E',
'spotify:artist:00tVTdpEhQQw1bqdu8RCx2',
'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb',
'spotify:artist:3dRfiJ2650SZu6GbydcHNb',
'spotify:artist:3b8QkneNDz4JHKKKlLgYZg',
'spotify:artist:6loBF9iQdE11WSX29fNKqY',
'spotify:artist:49bzE5vRBRIota4qeHtQM8',
'spotify:artist:0cQuYRSzlItquYxsQKDvVc',
'spotify:artist:7B4hKK0S9QYnaoqa9OuwgX',
'spotify:artist:3z3g65U7mmyyBmmDfsQK9x',
'spotify:artist:2B4ZHz4QDWJTXPFPgO5peE',
'spotify:artist:4pmNqFu71wEmFXSsYAiWK8',
'spotify:artist:18jZvCsW1PJ4FDQ5gEXuKp',
'spotify:artist:72OaDtakiy6yFqkt4TsiFt',
'spotify:artist:3RTzAwFprBqiskp550eSJX',
'spotify:artist:53A0W3U0s8diEn9RhXQhVz',
'spotify:artist:6fOMl44jA4Sp5b9PpYCkzz',
'spotify:artist:4tX2TplrkIP4v05BNC903e',
'spotify:artist:2UZMlIwnkgAEDBsw1Rejkn',
'spotify:artist:1RP2UpEaRzkF0Id3JigqD8',
'spotify:artist:0Xi59sEw38vRvwleSAVqoo',
'spotify:artist:5PSWc8Y94zFsAtZlTe7ipI',
'spotify:artist:69Fk1s1lzINOKqoSWjjEiE',
'spotify:artist:0DchahWJGQqrqr8PMM5zQD',
'spotify:artist:2OpqcUtj10HHvGG6h9VYC5',
'spotify:artist:5l8VQNuIg0turYE1VtM9zV',
'spotify:artist:2IU5UBup85LmAXh5U36DeC',
'spotify:artist:6XpaIBNiVzIetEPCWDvAFP',
'spotify:artist:4UuoJfJ9UybJft7a8E6UHX',
'spotify:artist:6evKD5JWJON3qPBJtUEmtY',
'spotify:artist:4DFhHyjvGYa9wxdHUjtDkc',
'spotify:artist:7xkAwz0bQTGDSbkofyQt3U',
'spotify:artist:6roFdX1y5BYSbp60OTJWMd',
'spotify:artist:3bGXaFVQLASmDMdjjeJr8a',
'spotify:artist:4mxWe1mtYIYfP040G38yvS',
'spotify:artist:0svyIX7Xu3DVMkrOtB88i6',
'spotify:artist:0xxJO75NeIAF5jYruLYIDT',
'spotify:artist:78OAtwN3XpjMcjibnF2Kw5',
'spotify:artist:4MPkNgar5uTd8Sqvrr7par',
'spotify:artist:5vngPClqofybhPERIqQMYd',
'spotify:artist:24GaH9tRBgZjlvOhpFuKi2',
'spotify:artist:0zOcE3mg9nS6l3yxt1Y0bK',
'spotify:artist:04gDigrS5kc9YWfZHwBETP',
'spotify:artist:6xTk3EK5T9UzudENVvu9YB',
'spotify:artist:4jWGfUCFeTu5e0wprRhHXR',
'spotify:artist:5e4Dhzv426EvQe3aDb64jL',
'spotify:artist:0aqrM7Fo3Ml66Kc5JAzmjt',
'spotify:artist:25NQNriVT2YbSW80ILRWJa',
'spotify:artist:25IG9fa7cbdmCIy3OnuH57',
'spotify:artist:2d3VHzlOEwXvmBdS4pzOPL',
'spotify:artist:0IdYRFTswLdsGwSnzOaGNF',
'spotify:artist:0HLOP0AdsaWe5Pt5TNueGC',
'spotify:artist:1SX44N5qjWuFcCK8WTO0c6',
'spotify:artist:4BclNkZtAUq1YrYNzye3N7',
'spotify:artist:3P33qFNGBVXl86yQYWspFj',
'spotify:artist:6TZdvF1kFzwnQLgHQynzsO',
'spotify:artist:3EhbVgyfGd7HkpsagwL9GS',
'spotify:artist:7r8RF1tN2A4CiGEplkp1oP',
'spotify:artist:7LnaAXbDVIL75IVPnndf7w',
'spotify:artist:1yAwtBaoHLEDWAnWR87hBT',
'spotify:artist:3OsRAKCvk37zwYcnzRf5XF',
'spotify:artist:5ZoVY6c0cokQrn448eRqSc',
'spotify:artist:4wLXwxDeWQ8mtUIRPxGiD6',
'spotify:artist:2CvCyf1gEVhI0mX6aFXmVI',
'spotify:artist:5rScKX1Sh1U67meeUyTGwk',
'spotify:artist:3cdvlbCYf4WoBdy89RWkNS',
'spotify:artist:5Rsz3E1aovbqsmLQOxgK2y',
'spotify:artist:0MCU2OpgnSB7rm5UPUMHgt',
'spotify:artist:2ziB7fzrXBoh1HUPS6sVFn',
'spotify:artist:7m60UAnbgFFNuJbmS6OxTk',
'spotify:artist:2sil8z5kiy4r76CRTXxBCA',
'spotify:artist:3Ngh2zDBRPEriyxQDAMKd1',
'spotify:artist:4ytkhMSAnrDP8XzRNlw9FS',
'spotify:artist:4xtWjIlVuZwTCeqVAsgEXy',
'spotify:artist:3DkkiE0G5N1d1AOG6yTMOo',
'spotify:artist:2xDc4cU0Rj6YG8SrmAHIy0',
'spotify:artist:1RMjtNqKOM6V36EngDqEBw',
'spotify:artist:0c173mlxpT3dSFRgMO8XPh',
'spotify:artist:0f3EsoviYnRKTkmayI3cux',
'spotify:artist:45KpBChrKYWhTlU6N57ofu',
'spotify:artist:7ihLzUpuNecU5VBkvOUDNq',
'spotify:artist:28pS8WVbFstY0o1SrqCf8I',
'spotify:artist:77tBvvyd6SD4Y9Um1xcbxP',
'spotify:artist:7HL4id2U7FSDJtfKQHMgQx',
'spotify:artist:0dgnITyIAN4NrmUVisvxHU',
'spotify:artist:3tufWJzpCiAGleBt5TkmTn',
'spotify:artist:3nLYJvqbEzs5kg2TlprxIG',
'spotify:artist:09hNSPPOxDop4FRdr6UEnq',
'spotify:artist:2qQeKHrQJHLLbvDAOSO874',
'spotify:artist:3y24n3XhZ96wgwRXjvS17T',
'spotify:artist:3eqjTLE0HfPfh78zjh6TqT',
'spotify:artist:5W5bDNCqJ1jbCgTxDD0Cb3',
'spotify:artist:5W5bDNCqJ1jbCgTxDD0Cb3',
'spotify:artist:6e7QpHYqEiyJGiM98IysLa',
'spotify:artist:3USpNaxpX1iNqNnQWqg9ob',
'spotify:artist:0b2XeWDPeBiLeskT6RFqMb',
'spotify:artist:3zNM2tRfTX6LI1lN2PlrTt',
'spotify:artist:1bBZcz4jP7CoPlqpCFh4gz',
'spotify:artist:5sAg1HZePcFfhrs0G8A8OP',
'spotify:artist:6OQrOpxSIfPai3cFaN4v4P',
'spotify:artist:4tujQJicOnuZRLiBFdp3Ou',
'spotify:artist:3WaJSfKnzc65VDgmj2zU8B',
'spotify:artist:6lE1ly8K8H7u8k2ej2plvv',
'spotify:artist:3537Phr9t8GWkY2GtxDpnt',
'spotify:artist:2RE8NwNxsOyuNZDD0jRxHP',
'spotify:artist:10n1KB2sjTrGdyuC83y8jW',
'spotify:artist:5kiBy7FO5L4ywMz1xF70PX',
'spotify:artist:60df5JBRRPcnSpsIMxxwQm',
'spotify:artist:1eYhYunlNJlDoQhtYBvPsi',
'spotify:artist:454VBUMshEg0sxzoWKTYbT',
'spotify:artist:375zxMmh2cSgUzFFnva0O7',
'spotify:artist:0fA0VVWsXO9YnASrzqfmYu',
'spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc',
'spotify:artist:5fikk4h5qbEebqK2Fc6e48',
'spotify:artist:6DPYiyq5kWVQS4RGwxzPC7',
'spotify:artist:4YveAIZNQZBiWbt7iWsvCa',
'spotify:artist:1G1mX30GpUJqOr1QU2eBSs',
'spotify:artist:762310PdDnwsDxAQxzQkfX',
'spotify:artist:0NKDgy9j66h3DLnN8qu1bB',
'spotify:artist:0z5DFXmhT4ZNzWElsM7V89',
'spotify:artist:2p4FqHnazRucYQHyDCdBrJ',
'spotify:artist:3vAaWhdBR38Q02ohXqaNHT',
'spotify:artist:4uN3DsfENc7dp0OLO0FEIb',
'spotify:artist:2Hkut4rAAyrQxRdof7FVJq',
'spotify:artist:3tJoFztHeIJkJWMrx0td2f',
'spotify:artist:3ipn9JLAPI5GUEo4y4jcoi',
'spotify:artist:4Js9eYwAf9rypNtV8pNSw9',
'spotify:artist:6AgTAQt8XS6jRWi4sX7w49',
'spotify:artist:757aE44tKEUQEqRuT6GnEB',
'spotify:artist:4Ga1P7PMIsmqEZqhYZQgDo',
'spotify:artist:13y7CgLHjMVRMDqxdx0Xdo',
'spotify:artist:3qfrrrSO7utFdJkM2tvMRb',
'spotify:artist:12Kgt2eahvxNWhD5PnSUde',
'spotify:artist:3WNUkxJcJeliFx9KXWXMgs',
'spotify:artist:5ndkK3dpZLKtBklKjxNQwT',
'spotify:artist:55Aa2cqylxrFIXC767Z865',
'spotify:artist:4OBJLual30L7gRl5UkeRcT',
'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i',
'spotify:artist:3ciRvbBIVz9fBoPbtSYq4x',
'spotify:artist:7d3WFRME3vBY2cgoP38RDo',
'spotify:artist:2kCcBybjl3SAtIcwdWpUe3',
'spotify:artist:5LfGQac0EIXyAN8aUwmNAQ',
'spotify:artist:5LfGQac0EIXyAN8aUwmNAQ',
'spotify:artist:5eyMzR1hYiEZtN2c9ly2kw',
'spotify:artist:3bhu7P5PfngueRHiB9hjcx',
'spotify:artist:0RqtSIYZmd4fiBKVFqyIqD',
'spotify:artist:1CQNVk80j5vFm9HXXPWCPS',
'spotify:artist:7AQJ6lwVCcej6eKZXQZus4',
'spotify:artist:2mxe0TnaNL039ysAj51xPQ',
'spotify:artist:3iDBygQ5jUSGX9N4NYIOPy',
'spotify:artist:3NkUVbGU8HjV9CqzfmtwMa',
'spotify:artist:6xXKQ1PWrstWHd7eH4z9lV',
'spotify:artist:2z16cLWVwv6yhVZil2tRq8',
'spotify:artist:29Wx2yXNi79vitQ4T1WQth',
'spotify:artist:5L49nCO2aBoY8H2wAMAgEj',
'spotify:artist:2RBLAeKfQpmzwWRFLakVRe',
'spotify:artist:5WkzBrXluz1sqLnz0EIHAf',
'spotify:artist:2TOripE27hynVXGqO1ilyl',
'spotify:artist:0SzEp7uVyzBozMmx2zx6TB',
'spotify:artist:0wFCfi4dqZB5DO7sAQpQUZ',
'spotify:artist:0zyyO1rbhrlJmPVXbOJGqZ',
'spotify:artist:3R0dLbQvI6x16t2bLaE9wE',
'spotify:artist:4jtcZjpuoSYBWP8xoXGi2P',
'spotify:artist:4UUA0PUMRTJsfa3eqQGY0T',
'spotify:artist:6lDJOI3SbOaTNvQsywhoz9',
'spotify:artist:77yIRk1rQ3x2bFWzvL6ri7',

]

In [62]:
_related=[]
for node_artist in _related_art:
    artist_related_artist_obj = sp.artist_related_artists(node_artist)['artists']
    for artist in artist_related_artist_obj:
        edge_artist = artist['name']
        followers = artist['followers']['total']
        genres = '; '.join(artist['genres'])
        popularity = artist['popularity']
        _type = artist['type']
        edge_uri = artist['uri']

        _related.append([node_artist,edge_artist,followers,genres,popularity,_type,edge_uri ])
    time.sleep(0.25) # Trying not to make spotify mad.

In [64]:
column_headers = ['node_artist','edge_artist','followers','genres','popularity','_type','edge_uri']

write_list_of_lists('data/spotify_artist_related_artists4.csv',_related,column_headers)

True

In [ ]:
artist_obj = sp.artist(artist)

In [8]:
sp.artist(_related_art[0])

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7urq0VfqxEYEEiZUkebXT4'},
 'followers': {'href': None, 'total': 1003390},
 'genres': ['atl hip hop',
  'boy band',
  'contemporary r&b',
  'dance pop',
  'gangster rap',
  'hip pop',
  'pop rap',
  'r&b',
  'southern hip hop',
  'urban contemporary'],
 'href': 'https://api.spotify.com/v1/artists/7urq0VfqxEYEEiZUkebXT4',
 'id': '7urq0VfqxEYEEiZUkebXT4',
 'images': [{'height': 130,
   'url': 'https://i.scdn.co/image/21a8b23a23a41f3736b2dd6a5df2d7dc8341ea05',
   'width': 200},
  {'height': 42,
   'url': 'https://i.scdn.co/image/7c1bc55302710bb4e426aa9581a0b87296dadfd8',
   'width': 64}],
 'name': '112',
 'popularity': 65,
 'type': 'artist',
 'uri': 'spotify:artist:7urq0VfqxEYEEiZUkebXT4'}

In [11]:
_artists =[]
for artist in _related_art:
    artist_obj = sp.artist(artist)
    try:
        followers = artist_obj['followers']['total']
    except IndexError:
        followers = ''
    try:
        genres = '; '.join(artist_obj['genres'])
    except IndexError:
        genres=''
    try:
        name = artist_obj['name']
    except IndexError:
        name=''
    try:
        popularity = artist_obj['popularity']
    except IndexError:
        popularity=''
    try:
        _artist_uri = artist_obj['uri']
    except IndexError:
        _artist_uri = ''
    _artists.append([name, popularity, followers,genres, _artist_uri  ])
    time.sleep(0.25) # Trying not to make spotify mad.

In [ ]:
column_headers = ['name', 'popularity', 'followers','genres', '_artist_uri'  ]
write_list_of_lists('data/spotify_artists4.csv',_artists)

In [65]:
sp.search('spotify:artist:7urq0VfqxEYEEiZUkebXT4',type='artist')['artists']['items']

KeyError: 'artists'

In [ ]:
spotify:artist:7urq0VfqxEYEEiZUkebXT4

In [69]:
# dir(sp)
sp.artists?

Signature: sp.artists(artists)
Docstring:
returns a list of artists given the artist IDs, URIs, or URLs

Parameters:
    - artists - a list of  artist IDs, URIs or URLs
File:      c:\users\ja238f\.pyenv\pyenv-win\versions\3.10.6\lib\site-packages\spotipy\client.py
Type:      method


In [75]:

artist_obj = sp.artist(artist)

In [76]:
artist_obj

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7urq0VfqxEYEEiZUkebXT4'},
 'followers': {'href': None, 'total': 1002610},
 'genres': ['atl hip hop',
  'boy band',
  'contemporary r&b',
  'dance pop',
  'gangster rap',
  'hip pop',
  'pop rap',
  'r&b',
  'southern hip hop',
  'urban contemporary'],
 'href': 'https://api.spotify.com/v1/artists/7urq0VfqxEYEEiZUkebXT4',
 'id': '7urq0VfqxEYEEiZUkebXT4',
 'images': [{'height': 130,
   'url': 'https://i.scdn.co/image/21a8b23a23a41f3736b2dd6a5df2d7dc8341ea05',
   'width': 200},
  {'height': 42,
   'url': 'https://i.scdn.co/image/7c1bc55302710bb4e426aa9581a0b87296dadfd8',
   'width': 64}],
 'name': '112',
 'popularity': 65,
 'type': 'artist',
 'uri': 'spotify:artist:7urq0VfqxEYEEiZUkebXT4'}

In [3]:
num = (3/7)*(5/9)

In [4]:
dom = (3/7)*(5/9) + (1/5*4/7)

In [5]:
num/dom

0.6756756756756757